## Crossroad

This IPython notebook file seems to be about creating "crossroad polygons" from geospatial street data that is used in **00_C_creating_and_oregenzing_OSM 01**. It involves loading street network data, identifying different shapes like triangles and rectangles in the network, buffering those shapes to make polygons, and filtering the results to get clean crossroad polygons.  

Here are the files it uses:

    csv_tables/TA_streets_20240724_031704/Streets.shp: Geospatial data for streets.

    os_ta_streets_nodes: Node data for the street network.

    os_ta_streets_edges: Edge data for the street network.

    edges_rectangles: Rectangle shapes identified in the street network.

    edges_triangles: Triangle shapes identified in the street network.

    edges_roundabout: Roundabout shapes in the street network.

    edges_rest: Remaining edges in the street network.

    ta_poly_crossroads: Crossroad polygons.

    ta_crossroads_08012025: Processed crossroad data.   

The notebook reads these files, processes the data, and creates polygons by buffering. Along the way, it visualizes the data using maps and also uses tools like leafmap to create interactive maps for better understanding and analysis.

In [46]:
import matplotlib.pyplot as plt
from shapely.geometry import MultiLineString, LineString

from shapely.geometry import Point
from shapely.ops import unary_union
import leafmap

import networkx as nx
import osmnx as ox
import geopandas as gpd
import pandas as pd
import numpy as np
from IPython.display import Image, display

List of files used:
* csv_tables/TA_streets_20240724_031704/Streets.shp
* os_ta_streets_nodes
* os_ta_streets_edges
* edges_rectangles
* edges_triangles
* edges_roundabout
* edges_rest
* ta_poly_crossroads
* ta_crossroads_08012025 number is date of export

In [47]:
# pd.set_option("display.max_rows", 5)


In [48]:
cols_to_export = ['os_ta_index', 'u', 'v', 'osmid', 'name', 'reversed', 'length'
                  , 'tunnel', 'bridge', 'junction', 'name_type', 'osmid_type',
       'name_fixed', 'ta_name', 'overlapping_names', 'overlapping_osmids',
       'overlapping_names_len', 'start_edge_idx', 'end_edge_idx', 'start_name',
       'end_name']

### Loading relevant data frames

In [49]:
# G = ox.graph_from_place("Tel Aviv, Israel", network_type="drive")
# # you can convert your graph to node and edge GeoPandas GeoDataFrames
# os_ta_streets_nodes, _ = ox.graph_to_gdfs(G)

# os_ta_streets_nodes = os_ta_streets_nodes.to_crs('32636')

# os_ta_streets_nodes = os_ta_streets_nodes.reset_index()
# os_ta_streets_nodes

In [50]:
os_ta_streets_nodes = gpd.read_parquet('./csv_tables/os_ta_streets_nodes.parquet')
os_ta_streets_nodes

,osmid,y,x,highway,street_count,ref,geometry,is_roundabout
0,139693,32.093840,34.790572,traffic_signals,4,None,POINT (668968.683 3552240.237),0
1,139698,32.093869,34.791231,None,3,None,POINT (669030.815 3552244.552),0
2,139707,32.095354,34.778500,None,3,None,POINT (667826.578 3552389.242),0
3,139708,32.095052,34.778329,None,3,None,POINT (667810.983 3552355.494),0
4,139709,32.094527,34.778842,None,4,None,POINT (667860.387 3552298.098),0
...,...,...,...,...,...,...,...,...
6483,12292683832,32.062786,34.785004,None,1,None,POINT (668500.151 3548788.711),0
6484,12292683834,32.063149,34.785276,traffic_signals,3,None,POINT (668525.204 3548829.326),0
6485,12361383714,32.057552,34.763578,None,1,None,POINT (666486.760 3548175.193),0
6486,12361652364,32.054494,34.771553,None,1,None,POINT (667245.361 3547848.419),0


In [51]:
os_ta_streets_nodes.head()


,osmid,y,x,highway,street_count,ref,geometry,is_roundabout
0,139693,32.093840,34.790572,traffic_signals,4,None,POINT (668968.683 3552240.237),0
1,139698,32.093869,34.791231,None,3,None,POINT (669030.815 3552244.552),0
2,139707,32.095354,34.778500,None,3,None,POINT (667826.578 3552389.242),0
3,139708,32.095052,34.778329,None,3,None,POINT (667810.983 3552355.494),0
4,139709,32.094527,34.778842,None,4,None,POINT (667860.387 3552298.098),0


In [52]:
os_ta_streets_edges = gpd.read_parquet('./csv_tables/os_ta_streets_edges.parquet')
os_ta_streets_edges.head(3)

,os_ta_index,geometry,u,v,osmid,name,reversed,length,tunnel,bridge,...,name_fixed,ta_name,overlapping_names,overlapping_osmids,overlapping_names_len,start_edge_idx,end_edge_idx,start_name,end_name,is_connected_to_rab
0,0,"LINESTRING (668968.683 3552240.237, 668968.629...",139693,5723720351,5118378,ויצמן,False,4.37,,,...,ויצמן,ויצמן,,,0,1,8725,יהודה המכבי,יהודה המכבי,0
1,1,"LINESTRING (668968.683 3552240.237, 668972.601...",139693,139698,167691710,יהודה המכבי,False,62.173,,,...,יהודה המכבי,יהודה המכבי,,,0,0,2,ויצמן,יהודה המכבי,0
2,2,"LINESTRING (669030.815 3552244.552, 669082.911...",139698,139723,167691710,יהודה המכבי,False,110.029,,,...,יהודה המכבי,יהודה המכבי,,,0,1,23,יהודה המכבי,יהודה המכבי,0


In [53]:
ta_streets = gpd.read_file('./csv_tables/TA_streets_20240724_031704/Streets.shp')
ta_streets

,oidrechov,krechov,trechov,shemangli,mslamas,tsug,kkivun,UniqueId,shemarvit,kreka,geometry
0,1.0,915.0,הרוגי מלכות,HARUGEY MALKHOT,336.0,רחוב,0.0,507-10001,قتل مملكة,100.0,"LINESTRING (672865.880 3554095.253, 672895.216..."
1,2.0,0.0,0,UKNOWN,0.0,רחוב,3.0,507-10002,None,100.0,"LINESTRING (666990.498 3551436.940, 667065.337..."
2,3.0,265.0,אמסטרדם,AMSTERDAM,516.0,רחוב,1.0,507-10003,أمستردام,100.0,"LINESTRING (667879.712 3551424.162, 667940.741..."
3,4.0,644.0,אלון יגאל,YIG'AL ALLON,2524.0,רחוב,0.0,507-10004,ألون ييغال,200.0,"LINESTRING (669570.036 3550420.535, 669581.404..."
4,5.0,634.0,מרגולין,MARGOLIN,2649.0,רחוב,1.0,507-10005,مارغولين,100.0,"LINESTRING (669329.153 3548322.758, 669409.403..."
...,...,...,...,...,...,...,...,...,...,...,...
8874,9851.0,3007.0,שבטי ישראל,SHIVTEY YISRA'EL,1983.0,רחוב,0.0,507-17843,قبائل إسرائيل,100.0,"LINESTRING (665771.816 3547023.159, 665760.256..."
8875,9852.0,3058.0,אבינרי יצחק,AVINERY,2027.0,רחוב,0.0,507-20562,Avinri Yitzhak,100.0,"LINESTRING (665585.719 3547178.152, 665627.936..."
8876,9853.0,3058.0,אבינרי יצחק,AVINERY,2027.0,רחוב,0.0,507-20563,Avinri Yitzhak,100.0,"LINESTRING (665700.142 3547064.296, 665759.119..."
8877,9855.0,3907.0,3907,None,1703.0,רחוב,0.0,507-21960,3907,100.0,"LINESTRING (665087.059 3546677.092, 665075.120..."


In [54]:
os_ta_streets_edges

,os_ta_index,geometry,u,v,osmid,name,reversed,length,tunnel,bridge,...,name_fixed,ta_name,overlapping_names,overlapping_osmids,overlapping_names_len,start_edge_idx,end_edge_idx,start_name,end_name,is_connected_to_rab
0,0,"LINESTRING (668968.683 3552240.237, 668968.629...",139693,5723720351,5118378,ויצמן,False,4.37,,,...,ויצמן,ויצמן,,,0,1,8725,יהודה המכבי,יהודה המכבי,0
1,1,"LINESTRING (668968.683 3552240.237, 668972.601...",139693,139698,167691710,יהודה המכבי,False,62.173,,,...,יהודה המכבי,יהודה המכבי,,,0,0,2,ויצמן,יהודה המכבי,0
2,2,"LINESTRING (669030.815 3552244.552, 669082.911...",139698,139723,167691710,יהודה המכבי,False,110.029,,,...,יהודה המכבי,יהודה המכבי,,,0,1,23,יהודה המכבי,יהודה המכבי,0
3,3,"LINESTRING (667826.578 3552389.242, 667810.983...",139707,139708,26516058,ירמיהו,False,37.249,,,...,ירמיהו,ירמיהו הנביא,,,0,4,5,אוסישקין,ירמיהו הנביא,0
4,4,"LINESTRING (667826.578 3552389.242, 667831.045...",139707,10985355495,1183058410,אוסישקין,False,190.227,,,...,אוסישקין,אוסישקין,,,0,3,9372,ירמיהו הנביא,אוסישקין,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9517,9523,"LINESTRING (668470.214 3547868.809, 668469.837...",12287814424,544561044,"[396627330, 396627331, 396627332, 139744099, 1...",איילון דרום,False,1158.0299999999997,,yes,...,איילון דרום,איילון דרום,,,0,93,4375,איילון דרום,חיל השריון,0
9518,9524,"LINESTRING (668470.214 3547868.809, 668467.408...",12287814424,2213119636,1082074722,,False,656.7250000000001,,,...,,"איילון דרום, כביש ירושלים תל אביב",,,0,93,96,איילון דרום,כביש ירושלים תל אביב,0
9519,9525,"LINESTRING (668470.453 3548624.735, 668458.737...",12292683830,1790662878,167691679,המסגר,False,136.733,,,...,המסגר,המסגר,,,0,500,6863,המסגר,המסגר,0
9520,9526,"LINESTRING (668525.204 3548829.326, 668538.677...",12292683834,6145368549,1106792626,הרכב,False,31.335,,,...,הרכב,הרכב,,,0,5582,2535,המסגר,הרכב,0


### Creating Crossroads Polygons

In [55]:
import geopandas as gpd
from shapely.geometry import LineString

def split_linestrings_by_length_return_two_gdfs(gdf, split_length=5):
    """
    Splits each LineString in a GeoDataFrame into three segments:
      1) Start to split_length
      2) split_length to (total_length - split_length)
      3) (total_length - split_length) to end

    Returns two GeoDataFrames:
      - ends_gdf: contains the start and end segments
      - middle_gdf: contains the middle segment

    Only splits if 2 * split_length < the total LineString length.
    """
    ends_rows = []
    middle_rows = []
    
    for _, row in gdf.iterrows():
        geometry = row.geometry
        
        # Check if geometry is a single LineString
        if isinstance(geometry, LineString):
            length = geometry.length
            
            # Ensure the line is long enough to split
            if split_length * 2 >= length:
                # If it's not long enough, skip or optionally just keep it as-is
                continue
            
            # Interpolate split points
            split_point_start = geometry.interpolate(split_length)
            split_point_end = geometry.interpolate(length - split_length)
            
            # Create three segments
            first_segment = LineString([
                geometry.coords[0],
                split_point_start.coords[0]
            ])
            middle_segment = LineString([
                split_point_start.coords[0],
                split_point_end.coords[0]
            ])
            last_segment = LineString([
                split_point_end.coords[0],
                geometry.coords[-1]
            ])
            
            # Convert row to a dict for copying attributes
            row_dict = row.to_dict()
            
            # Append first and last segments to ends_rows
            ends_rows.append({**row_dict, "geometry": first_segment})
            ends_rows.append({**row_dict, "geometry": last_segment})
            
            # Append middle segment to middle_rows
            middle_rows.append({**row_dict, "geometry": middle_segment})
    
    # Build GeoDataFrames
    ends_gdf = gpd.GeoDataFrame(ends_rows, crs=gdf.crs)
    middle_gdf = gpd.GeoDataFrame(middle_rows, crs=gdf.crs)
    
    return ends_gdf, middle_gdf


### trying getting only edges and nodes that are in a triangle shape.

General plan to getting these nodes/edges.

In the edges data frame I have the information of the nodes in columns u, v.<br>

A closed triangle has:
nodes: a, b, c
edges: 1, 2, 3

* group edges by v (end edge)
* the result will be grouped again for edges that their u edge (beginning edge) == v
* and from that result the if there exist an edge that has  


In [56]:
df = os_ta_streets_edges.copy()
df.shape

(9522, 23)

In [57]:
import collections

# "adj" will map each node -> set of its neighbors (undirected)
adj = collections.defaultdict(set)

for row in df.itertuples():
    # row.u, row.v from the GeoDataFrame
    a, b = row.u, row.v

    # If this is truly undirected, add each way:
    adj[a].add(b)
    adj[b].add(a)


In [58]:
triangles = set()

for a in adj:
    # Consider pairs (b,c) of neighbors of a
    for b in adj[a]:
        if b == a:
            continue
        for c in adj[a]:
            if c == a or c <= b:
                # Avoid duplicates, e.g. (b,c) and (c,b)
                continue
            # Check if b and c are directly connected
            if c in adj[b]:
                tri_nodes = tuple(sorted([a, b, c]))
                triangles.add(tri_nodes)

# "triangles" is a set of 3-node tuples: {('a','b','c'), ('x','y','z'), ...}


In [59]:
def get_edges_for_triangle(tri, edges_gdf):
    """Return the rows (edges) in edges_gdf that connect the 3 nodes in tri."""
    a, b, c = tri

    # We want edges for pairs (a,b), (a,c), (b,c), in either orientation
    mask = (
        ((edges_gdf["u"] == a) & (edges_gdf["v"] == b)) |
        ((edges_gdf["u"] == b) & (edges_gdf["v"] == a)) |
        
        ((edges_gdf["u"] == a) & (edges_gdf["v"] == c)) |
        ((edges_gdf["u"] == c) & (edges_gdf["v"] == a)) |
        
        ((edges_gdf["u"] == b) & (edges_gdf["v"] == c)) |
        ((edges_gdf["u"] == c) & (edges_gdf["v"] == b))
    )

    return edges_gdf[mask]


In [60]:
triangle_to_edges = {}

for tri in triangles:
    sub_gdf = get_edges_for_triangle(tri, df)
    # sub_gdf is still a GeoDataFrame, with geometry & CRS
    triangle_to_edges[tri] = sub_gdf


all_rows = []
for tri in triangles:
    sub_gdf = get_edges_for_triangle(tri, df).copy()
    sub_gdf["triangle"] = [tri] * len(sub_gdf)
    all_rows.append(sub_gdf)

df_triangles = pd.concat(all_rows, ignore_index=True)

# Make sure it is recognized as a GeoDataFrame
df_triangles = gpd.GeoDataFrame(df_triangles, geometry="geometry", crs=df.crs)
df_triangles

,os_ta_index,geometry,u,v,osmid,name,reversed,length,tunnel,bridge,...,ta_name,overlapping_names,overlapping_osmids,overlapping_names_len,start_edge_idx,end_edge_idx,start_name,end_name,is_connected_to_rab,triangle
0,9316,"LINESTRING (668786.467 3550522.050, 668787.034...",10790642178,10790642196,1215824332,ויצמן,False,20.119,,,...,ויצמן,,,0,5122,9319,ויצמן,דפנה,0,"(10790642178, 10790642194, 10790642196)"
1,9318,"LINESTRING (668794.328 3550516.059, 668790.104...",10790642194,10790642178,1215824333,ויצמן,False,9.998,,,...,ויצמן,,,0,277,5122,ויצמן,ויצמן,0,"(10790642178, 10790642194, 10790642196)"
2,9319,"LINESTRING (668794.328 3550516.059, 668790.553...",10790642194,10790642196,1215824334,דפנה,False,15.849,,,...,דפנה,,,0,277,9316,ויצמן,ויצמן,0,"(10790642178, 10790642194, 10790642196)"
3,9433,"LINESTRING (669705.256 3557647.098, 669698.098...",11267490914,11267490925,1216081164,פרופ' פנינה זלצמן,False,35.005,,,...,זלצמן פנינה פרופ',,,0,9432,7933,זלצמן פנינה פרופ',זלצמן פנינה פרופ',0,"(11267490914, 11267490925, 11267490956)"
4,9434,"LINESTRING (669705.256 3557647.098, 669698.840...",11267490914,11267490956,1308836996,פרופ' פנינה זלצמן,False,18.094,,,...,זלצמן פנינה פרופ',,,0,9432,9435,זלצמן פנינה פרופ',זלצמן פנינה פרופ',0,"(11267490914, 11267490925, 11267490956)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2011,4794,"LINESTRING (669920.491 3553636.342, 669908.492...",918718913,1127659117,355307997,שלום רוזנפלד,False,23.933,,,...,רוזנפלד שלום,,,0,1122,5146,רוזנפלד שלום,לבנון חיים,0,"(1127659117, 1127659147, 918718913)"
2012,5149,"LINESTRING (669898.481 3553645.811, 669901.719...",1127659117,1127659147,97354492,חיים לבנון,False,21.948999999999998,,,...,לבנון חיים,,,0,4794,4793,רוזנפלד שלום,,0,"(1127659117, 1127659147, 918718913)"
2013,4684,"LINESTRING (666145.824 3548345.064, 666143.979...",724191521,6454784752,204138350,יחזקאל קויפמן,False,22.115000000000002,,,...,קויפמן יחזקאל,,,0,3122,7137,קויפמן יחזקאל,,0,"(1953545098, 6454784752, 724191521)"
2014,7137,"LINESTRING (666120.893 3548343.305, 666126.597...",1953545098,6454784752,184820083,,False,31.538000000000004,,,...,"גולדמן נחום, קויפמן יחזקאל",,,0,7138,4684,גולדמן נחום,קויפמן יחזקאל,0,"(1953545098, 6454784752, 724191521)"


In [61]:
os_ta_streets_nodes_cp = os_ta_streets_nodes.copy()
os_ta_streets_nodes_cp.geometry = os_ta_streets_nodes_cp.buffer(3)

In [62]:
# ## checking created polygons

# m = leafmap.Map(center=(32.047, 34.785), zoom=11)

# m.add_gdf(df_triangles)
# m.add_gdf(os_ta_streets_nodes_cp, fill_colors=['red'])


# m

List of edges to be excluded from the triangle since they are not crossroad:
* remove roundabouts
[6805, 5681, 9404, 9408,7595, 7596, 5981, 5213, 5212,3722, 3729, 3723, 6051, 6037, 6050, 6074, 6059, 6060, 6073, 6055, 3496, 6130, 6118, 6129,
 9253, 6034, 9399, 6104. 4811, 3878, 4812,5811,  8710, 8695, 8711, 9013, 9017, 4188, 4186, 5109, 5109, 5113, 5085, 5097, 5098, 5104, 7816, 7225, 7223
 5636, 1395, 5701, 5700, 5703, 9268, 4024, 4480, 4022, 4481]

**Too much work!**

-------

Let create polygons from our edges. then use the prior method of creating polygons by breaking and buffering
If a resulting polygon is not included in the triangle polygon then we remove these edges.


Plan:
* create triangle polygons
    * add column of tri_idx
    * get area
* Create medium crossroads polygons
    * add cross_idx
    * calculate area
* sjoin between cross_poly and os_ta_streets_edges
    * group by cross_idx
    * get unique os_ta_idx
* sjoin between cross_poly and tri_poly
    * group by tri_idx
    * count how many cross_poly fall in
    * calculate area share

In [63]:
df_tri_cp = df_triangles.copy()

#### create triangle polygons
df_tri_cp.geometry = df_tri_cp.buffer(6)

df_tri_cp_uni = unary_union(df_tri_cp.geometry)
df_tri_cp_uni = gpd.GeoDataFrame(geometry=[df_tri_cp_uni], crs=df_tri_cp.crs)

## explode
df_tri_cp_explode = df_tri_cp_uni.explode(index_parts=True).reset_index(drop=True)
df_tri_cp_explode.reset_index(inplace=True)

## adding idx column to later be used to filter 
df_tri_cp_explode.columns = ['tri_id','geometry']
df_tri_cp_explode['tri_area'] = df_tri_cp_explode.geometry.area

df_tri_cp_explode['tri_id'] = df_tri_cp_explode['tri_id'].apply(lambda x: str(x) + '_tri_id')
display(df_tri_cp_explode), df_tri_cp_explode.shape

#### creating cross road poly
os_ta_streets_edges_cp = os_ta_streets_edges.copy()
os_ta_streets_edges_split_ends, os_ta_streets_edges_split_middle = split_linestrings_by_length_return_two_gdfs(os_ta_streets_edges_cp, 6)
os_ta_streets_edges_split_ends.geometry = os_ta_streets_edges_split_ends.buffer(6)

os_ta_streets_edges_uni = unary_union(os_ta_streets_edges_split_ends.geometry)
os_ta_streets_edges_uni = gpd.GeoDataFrame(geometry=[os_ta_streets_edges_uni], crs=os_ta_streets_edges.crs)
## explode
cross_cp_explode = os_ta_streets_edges_uni.explode(index_parts=True).reset_index(drop=True)
cross_cp_explode.reset_index(inplace=True)

## adding idx column to later be used to filter 
cross_cp_explode.columns = ['cr_id','geometry']
cross_cp_explode['cr_area'] = cross_cp_explode.geometry.area

cross_cp_explode['cr_id'] = cross_cp_explode['cr_id'].apply(lambda x: str(x) + '_cr_id')
display(cross_cp_explode), cross_cp_explode.shape

,tri_id,geometry,tri_area
0,0_tri_id,"POLYGON ((664982.831 3545197.962, 664982.038 3...",195.587846
1,1_tri_id,"POLYGON ((665609.535 3545526.135, 665609.588 3...",926.351704
2,2_tri_id,"POLYGON ((664898.317 3545554.923, 664898.513 3...",1582.225711
3,3_tri_id,"POLYGON ((665458.800 3545544.147, 665458.872 3...",628.119004
4,4_tri_id,"POLYGON ((665410.377 3545548.979, 665410.356 3...",1102.913438
...,...,...,...
482,482_tri_id,"POLYGON ((669918.402 3557567.898, 669911.911 3...",3185.737946
483,483_tri_id,"POLYGON ((669509.772 3557628.619, 669510.010 3...",1281.317184
484,484_tri_id,"POLYGON ((669740.875 3557627.187, 669741.553 3...",688.631155
485,485_tri_id,"POLYGON ((669700.945 3557639.775, 669696.384 3...",806.610551


,cr_id,geometry,cr_area
0,0_cr_id,"POLYGON ((665227.766 3545034.839, 665227.297 3...",248.547432
1,1_cr_id,"POLYGON ((665172.306 3545070.751, 665172.300 3...",256.915585
2,2_cr_id,"POLYGON ((665128.917 3545098.534, 665128.871 3...",312.986751
3,3_cr_id,"POLYGON ((665224.575 3545132.051, 665225.046 3...",184.915746
4,4_cr_id,"POLYGON ((665175.387 3545165.416, 665175.540 3...",369.751486
...,...,...,...
4622,4622_cr_id,"POLYGON ((670426.140 3555099.150, 670426.574 3...",314.781310
4623,4623_cr_id,"POLYGON ((670698.740 3555095.330, 670698.694 3...",279.099380
4624,4624_cr_id,"POLYGON ((671532.324 3555096.831, 671532.209 3...",316.198848
4625,4625_cr_id,"POLYGON ((672117.040 3555099.764, 672117.010 3...",184.915746


(None, (4627, 3))

#### sjoin between cross_poly and os_ta_streets_edges
* group by cross_idx
* get unique os_ta_idx

In [64]:
cross_sjoin = gpd.sjoin(cross_cp_explode, os_ta_streets_edges, how='inner', predicate='intersects')
cross_sjoingb_os_id = cross_sjoin.groupby('cr_id').os_ta_index.unique()
# cross_sjoingb_os_id
cross_cp_explode['os_id'] = cross_cp_explode['cr_id'].map(lambda x: cross_sjoingb_os_id[x])
cross_cp_explode

,cr_id,geometry,cr_area,os_id
0,0_cr_id,"POLYGON ((665227.766 3545034.839, 665227.297 3...",248.547432,"[695, 9224]"
1,1_cr_id,"POLYGON ((665172.306 3545070.751, 665172.300 3...",256.915585,"[695, 659]"
2,2_cr_id,"POLYGON ((665128.917 3545098.534, 665128.871 3...",312.986751,"[659, 660, 661]"
3,3_cr_id,"POLYGON ((665224.575 3545132.051, 665225.046 3...",184.915746,[668]
4,4_cr_id,"POLYGON ((665175.387 3545165.416, 665175.540 3...",369.751486,"[660, 668, 669, 667]"
...,...,...,...,...
4622,4622_cr_id,"POLYGON ((670426.140 3555099.150, 670426.574 3...",314.781310,"[859, 860, 861]"
4623,4623_cr_id,"POLYGON ((670698.740 3555095.330, 670698.694 3...",279.099380,"[4363, 4364, 3443]"
4624,4624_cr_id,"POLYGON ((671532.324 3555096.831, 671532.209 3...",316.198848,"[2715, 2714, 2716]"
4625,4625_cr_id,"POLYGON ((672117.040 3555099.764, 672117.010 3...",184.915746,[3278]


<!-- #### sjoin between cross_poly and tri_poly
* group by tri_idx
* count how many cross_poly fall in
* calculate area share -->

In [65]:
from itertools import chain

In [66]:
cr_tri_joined = gpd.sjoin(cross_cp_explode, df_tri_cp_explode, how='inner', predicate='intersects')

cr_tri_gb = cr_tri_joined.groupby('tri_id').agg({
    'cr_id':  lambda g: g.count(),
    'cr_area':  lambda g: g.sum(),
    'os_id': lambda g: list(set(chain.from_iterable(g)))
}).reset_index()


cr_tri_gb.columns = ['tri_id','cr_cnt','cr_area', 'os_id']
df_tri_cp_explode['cr_cnt'] = df_tri_cp_explode['tri_id'].map(lambda x: cr_tri_gb[cr_tri_gb.tri_id == x]['cr_cnt'].values[0])
df_tri_cp_explode['cr_area'] = df_tri_cp_explode['tri_id'].map(lambda x: cr_tri_gb[cr_tri_gb.tri_id == x]['cr_area'].values[0])
df_tri_cp_explode['os_id'] = df_tri_cp_explode['tri_id'].map(lambda x: cr_tri_gb[cr_tri_gb.tri_id == x]['os_id'].values[0])


df_tri_cp_explode

,tri_id,geometry,tri_area,cr_cnt,cr_area,os_id
0,0_tri_id,"POLYGON ((664982.831 3545197.962, 664982.038 3...",195.587846,1,453.741343,"[6801, 662, 6802, 661, 6798, 670, 6799, 6800]"
1,1_tri_id,"POLYGON ((665609.535 3545526.135, 665609.588 3...",926.351704,1,917.941966,"[8967, 4267, 8966, 4547, 4264, 8965]"
2,2_tri_id,"POLYGON ((664898.317 3545554.923, 664898.513 3...",1582.225711,2,1213.170841,"[5685, 5682, 5681, 6805, 5683, 5680, 5684, 568..."
3,3_tri_id,"POLYGON ((665458.800 3545544.147, 665458.872 3...",628.119004,2,1254.037892,"[9408, 9404, 6804, 9405, 9403, 4263, 9407, 680..."
4,4_tri_id,"POLYGON ((665410.377 3545548.979, 665410.356 3...",1102.913438,2,1385.746949,"[9408, 9404, 6804, 9405, 9403, 7078, 691, 7077..."
...,...,...,...,...,...,...
482,482_tri_id,"POLYGON ((669918.402 3557567.898, 669911.911 3...",3185.737946,2,682.488258,"[5274, 2197, 3220, 5273]"
483,483_tri_id,"POLYGON ((669509.772 3557628.619, 669510.010 3...",1281.317184,3,1949.323965,"[9359, 9362, 9347, 9355, 9356, 9360, 9303, 935..."
484,484_tri_id,"POLYGON ((669740.875 3557627.187, 669741.553 3...",688.631155,2,1090.338586,"[9214, 9432, 9429, 9517, 9215, 9430, 9516, 943..."
485,485_tri_id,"POLYGON ((669700.945 3557639.775, 669696.384 3...",806.610551,2,831.682797,"[9434, 9436, 9432, 7933, 9433, 9435]"


In [67]:
df_tri_cp_explode['cr_share'] = df_tri_cp_explode['cr_area'] / df_tri_cp_explode['tri_area']
df_tri_cp_explode['cr_share'] = df_tri_cp_explode['cr_share'].apply(lambda x: round(x, 2))
df_tri_cp_explode

,tri_id,geometry,tri_area,cr_cnt,cr_area,os_id,cr_share
0,0_tri_id,"POLYGON ((664982.831 3545197.962, 664982.038 3...",195.587846,1,453.741343,"[6801, 662, 6802, 661, 6798, 670, 6799, 6800]",2.32
1,1_tri_id,"POLYGON ((665609.535 3545526.135, 665609.588 3...",926.351704,1,917.941966,"[8967, 4267, 8966, 4547, 4264, 8965]",0.99
2,2_tri_id,"POLYGON ((664898.317 3545554.923, 664898.513 3...",1582.225711,2,1213.170841,"[5685, 5682, 5681, 6805, 5683, 5680, 5684, 568...",0.77
3,3_tri_id,"POLYGON ((665458.800 3545544.147, 665458.872 3...",628.119004,2,1254.037892,"[9408, 9404, 6804, 9405, 9403, 4263, 9407, 680...",2.00
4,4_tri_id,"POLYGON ((665410.377 3545548.979, 665410.356 3...",1102.913438,2,1385.746949,"[9408, 9404, 6804, 9405, 9403, 7078, 691, 7077...",1.26
...,...,...,...,...,...,...,...
482,482_tri_id,"POLYGON ((669918.402 3557567.898, 669911.911 3...",3185.737946,2,682.488258,"[5274, 2197, 3220, 5273]",0.21
483,483_tri_id,"POLYGON ((669509.772 3557628.619, 669510.010 3...",1281.317184,3,1949.323965,"[9359, 9362, 9347, 9355, 9356, 9360, 9303, 935...",1.52
484,484_tri_id,"POLYGON ((669740.875 3557627.187, 669741.553 3...",688.631155,2,1090.338586,"[9214, 9432, 9429, 9517, 9215, 9430, 9516, 943...",1.58
485,485_tri_id,"POLYGON ((669700.945 3557639.775, 669696.384 3...",806.610551,2,831.682797,"[9434, 9436, 9432, 7933, 9433, 9435]",1.03


Checking different share to determine what os_id to take and then drop from df_triangle

In [68]:
print('shape of 0.25 below share',df_tri_cp_explode[df_tri_cp_explode.cr_share < 0.25].shape)
print('shape of 0.35 below share',df_tri_cp_explode[df_tri_cp_explode.cr_share < 0.35].shape)
print('shape of 0.45 below share',df_tri_cp_explode[df_tri_cp_explode.cr_share < 0.45].shape)
print('shape of 0.55 below share',df_tri_cp_explode[df_tri_cp_explode.cr_share < 0.55].shape)
print('shape of 0.65 below share',df_tri_cp_explode[df_tri_cp_explode.cr_share < 0.65].shape)




shape of 0.25 below share (38, 7)
shape of 0.35 below share (68, 7)
shape of 0.45 below share (85, 7)
shape of 0.55 below share (108, 7)
shape of 0.65 below share (129, 7)


In [69]:
df_tri_cp_explode[(df_tri_cp_explode.cr_share < 1.15) & (df_tri_cp_explode.cr_share >= 1.05)].shape

(24, 7)

In [70]:
# ## checking created polygons

# m = leafmap.Map(center=(32.047, 34.785), zoom=11)


# m.add_gdf(df_tri_cp_explode[(df_tri_cp_explode.cr_share < 1.15) & (df_tri_cp_explode.cr_share >= 1.05)])

# m

In [71]:
# below 0.65 to keep
tri_id_to_keep  =['332_tri_id','326_tri_id', '335_tri_id', '353_tri_id','405_tri_id', '442_tri_id','442_tri_id','424_tri_id','424_tri_id', '467_tri_id',
                  '472_tri_id','481_tri_id','484_tri_id','171_tri_id','20_tri_id', '15_tri_id', '9_tri_id', '293_tri_id','265_tri_id','323_tri_id',
                  '437_tri_id','446_tri_id','418_tri_id','483_tri_id','325_tri_id','501_tri_id','203_tri_id','168_tri_id','108_tri_id','84_tri_id','235_tri_id',
                  '67_tri_id','262_tri_id','337_tri_id','60_tri_id','327_tri_id']

tri_to_maybe_keep = ['46_tri_id']

# between 0.95 and 0.65 to drop
tri_id_to_drop = ['87_tri_id','97_tri_id','162_tri_id','26_tri_id','162_tri_id','225_tri_id','493_tri_id','238_tri_id','341_tri_id','376_tri_id','42_tri_id',
                  '77_tri_id','133_tri_id','222_tri_id','2_tri_id','92_tri_id','141_tri_id','423_tri_id','27_tri_id','59_tri_id','223_tri_id','268_tri_id',
                  '374_tri_id','1_tri_id','86_tri_id','185_tri_id','318_tri_id']

Getting os_id from list of drop and keep and preforming accordingly

In [72]:
os_id_keep_ls = df_tri_cp_explode[df_tri_cp_explode.tri_id.isin(tri_id_to_keep)].os_id.to_list()
os_id_keep_ls = list(set(chain.from_iterable(os_id_keep_ls)))
os_id_keep_ls
print('os_id_keep_ls',len(os_id_keep_ls))

os_id_drop_between_95_65_ls = df_tri_cp_explode[df_tri_cp_explode.tri_id.isin(tri_id_to_drop)].os_id.to_list()
os_id_drop_between_95_65_ls = list(set(chain.from_iterable(os_id_drop_between_95_65_ls)))
os_id_drop_between_95_65_ls
print('os_id_drop_between_95_65_ls',len(os_id_drop_between_95_65_ls))


os_id_drop_ls = df_tri_cp_explode[(df_tri_cp_explode.cr_share < 0.65)].os_id.to_list()
os_id_drop_ls = list(set(chain.from_iterable(os_id_drop_ls)))
os_id_drop_ls
print('os_id_drop_ls len:',len(os_id_drop_ls))


os_id_drop_ls = list(set(os_id_drop_ls + os_id_drop_between_95_65_ls))
# remove from os_id_drop_ls the item from os_id_keep_ls
print('os_id_drop_ls len:',len(os_id_drop_ls))

os_id_drop_ls = list(set(os_id_drop_ls) - set(os_id_keep_ls))
print('os_id_drop_ls len:',len(os_id_drop_ls))


os_id_keep_ls 351
os_id_drop_between_95_65_ls 241
os_id_drop_ls len: 1116
os_id_drop_ls len: 1310
os_id_drop_ls len: 1142


Making sure what I keep and what I drop make sense

In [73]:
# ## checking created polygons

# m = leafmap.Map(center=(32.047, 34.785), zoom=11)


# m.add_gdf(df_triangles[df_triangles.os_ta_index.isin(os_id_drop_ls)])
# m.add_gdf(df_triangles[df_triangles.os_ta_index.isin(os_id_keep_ls)], fill_colors='black')


# m

Dropping from triangle

There is a difference in number of os_id in drop list compared to what is in df_triangles<br>
This is because when creating crossroads we caught edges that weren't in df_triangles. 

In [74]:
df_triangles_clean = df_triangles[~df_triangles.os_ta_index.isin(os_id_drop_ls)]
# df_triangles_clean = df_triangles_clean[]
# df_triangles_clean


In [75]:
df_triangles_clean = df_triangles_clean[(df_triangles_clean.junction != 'roundabout') & (df_triangles_clean.is_connected_to_rab == 0)]

In [76]:
df_triangles_clean.to_parquet('./csv_tables/edges_triangles.parquet')

In [77]:
# st_cnt_2_edges_joined = unary_union(st_cnt_2_edges_buff.geometry)

# # Create polygons GeoDataFrame
# st_cnt_2_edges_joined = gpd.GeoDataFrame(geometry=[st_cnt_2_edges_joined], crs=st_cnt_2_edges_buff.crs)
# st_cnt_2_edges_joined.head(3), st_cnt_2_edges_joined.shape

# st_cnt_2_edges_explode = st_cnt_2_edges_joined.explode(index_parts=True).reset_index(drop=True)
# st_cnt_2_edges_explode.reset_index(inplace=True)


# st_cnt_2_edges_explode.columns = ['st_cnt_2_idx','geometry']

# st_cnt_2_edges_explode['st_cnt_2_idx'] = st_cnt_2_edges_explode['st_cnt_2_idx'].apply(lambda x: str(x) + '_idx')
# display(st_cnt_2_edges_explode), st_cnt_2_edges_explode.shape

# crossroad_st_cnt_2_name = crossroad_st_cnt_2.groupby('st_cnt_2_idx').ta_name.unique()
# crossroad_st_cnt_2_os_ta_index = crossroad_st_cnt_2.groupby('st_cnt_2_idx').os_ta_index.unique()


# crossroad_st_cnt_2_name.name = 'crossroad_name'

# crossroad_st_cnt_2_os_ta_index.name = 'os_edge_idx'

# # crossroad_st_cnt_2_os_ta_index.index.name
# display(crossroad_st_cnt_2_name)
# display(crossroad_st_cnt_2_os_ta_index)

# crossroad_st_cnt_2 = crossroad_st_cnt_2.drop_duplicates(subset=['st_cnt_2_idx'])

# crossroad_st_cnt_2_merged = crossroad_st_cnt_2.merge(crossroad_st_cnt_2_name, on='st_cnt_2_idx')
# crossroad_st_cnt_2_merged = crossroad_st_cnt_2_merged.merge(crossroad_st_cnt_2_os_ta_index, on='st_cnt_2_idx')

# crossroad_st_cnt_2_merged.shape

# def break_name(names):
#     new_names = []
#     for name in names:
#         if ',' in name:
#             split_name = name.split(',')
#             for s_name in split_name:
#                 strip_name = s_name.strip()
#                 new_names.append(strip_name)            
#         else:
#             new_names.append(name)
#     new_names = set(new_names)
#     new_names_ls = list(new_names)
#     return new_names_ls


# crossroad_st_cnt_2_merged.crossroad_name = crossroad_st_cnt_2_merged.crossroad_name.apply(break_name)
# crossroad_st_cnt_2_merged


# def clean_crossroad_name(names):
#     new_names = []
#     for name in names:
#         if len(name) > 1 and name != '':
#             new_names.append(name)
#     return new_names


# crossroad_st_cnt_2_merged.crossroad_name = crossroad_st_cnt_2_merged.crossroad_name.apply(clean_crossroad_name)
# crossroad_st_cnt_2_merged


# crossroad_st_cnt_2_merged['crossroad_name_len'] = crossroad_st_cnt_2_merged.crossroad_name.apply(lambda x: len(x))
# crossroad_st_cnt_2_merged['os_edge_idx_len'] = crossroad_st_cnt_2_merged.os_edge_idx.apply(lambda x: len(x))

# crossroad_st_cnt_2_merged

# crossroad_st_cnt_2_merged['area'] = crossroad_st_cnt_2_merged.area.apply(lambda x: round(x, 3))

#### Getting edges in a shape of a rectangle.

In [78]:
import collections

df = os_ta_streets_edges.copy()

# "adj" will map each node -> set of its neighbors (undirected)
adj = collections.defaultdict(set)

for row in df.itertuples():
    # row.u, row.v from the GeoDataFrame
    a, b = row.u, row.v
    
    # If this is truly undirected, add both directions
    adj[a].add(b)
    adj[b].add(a)


In [79]:
rectangles = set()

for a in adj:
    for b in adj[a]:
        # To avoid re-checking the same pair in reverse
        if b <= a:
            continue

        # Find common neighbors
        common_neighbors = adj[a].intersection(adj[b])
        if len(common_neighbors) < 2:
            continue
        
        # For each pair (c, d) among these neighbors, we form a rectangle
        cn_list = sorted(common_neighbors)
        for i in range(len(cn_list)):
            for j in range(i+1, len(cn_list)):
                c = cn_list[i]
                d = cn_list[j]
                
                # The rectangle is the set of nodes {a, b, c, d}
                # Sort them so we don't double-count the same 4-cycle
                rect_nodes = tuple(sorted([a, b, c, d]))
                rectangles.add(rect_nodes)

print("Number of rectangles found:", len(rectangles))


Number of rectangles found: 42


In [80]:
def get_edges_for_rectangle(rect, edges_gdf):
    """Return the rows (edges) in edges_gdf that connect the 4 nodes in rect."""
    a, b, c, d = rect  # rect is a tuple of 4 node IDs
    
    # We'll collect *all* edges that connect these 4 nodes pairwise.
    # This may return up to 6 edges (the complete subgraph on 4 nodes).
    # If you only want the 4 edges that form the actual cycle,
    # you might need additional logic to pick exactly the cycle edges.
    
    mask = (
        (((edges_gdf["u"] == a) & (edges_gdf["v"] == b)) | 
         ((edges_gdf["u"] == b) & (edges_gdf["v"] == a))) |

        (((edges_gdf["u"] == a) & (edges_gdf["v"] == c)) | 
         ((edges_gdf["u"] == c) & (edges_gdf["v"] == a))) |

        (((edges_gdf["u"] == a) & (edges_gdf["v"] == d)) | 
         ((edges_gdf["u"] == d) & (edges_gdf["v"] == a))) |

        (((edges_gdf["u"] == b) & (edges_gdf["v"] == c)) | 
         ((edges_gdf["u"] == c) & (edges_gdf["v"] == b))) |

        (((edges_gdf["u"] == b) & (edges_gdf["v"] == d)) | 
         ((edges_gdf["u"] == d) & (edges_gdf["v"] == b))) |

        (((edges_gdf["u"] == c) & (edges_gdf["v"] == d)) | 
         ((edges_gdf["u"] == d) & (edges_gdf["v"] == c)))
    )

    return edges_gdf[mask]


In [81]:
rectangle_to_edges = {}

for rect in rectangles:
    sub_gdf = get_edges_for_rectangle(rect, df)
    rectangle_to_edges[rect] = sub_gdf



all_rows = []
for rect in rectangles:
    sub_gdf = get_edges_for_rectangle(rect, df).copy()
    # Tag each row with the rectangle it belongs to
    sub_gdf["rectangle"] = [rect] * len(sub_gdf)
    all_rows.append(sub_gdf)

df_rectangles = pd.concat(all_rows, ignore_index=True)
df_rectangles = gpd.GeoDataFrame(df_rectangles, geometry="geometry", crs=df.crs)

print("df_rectangles shape:", df_rectangles.shape)
df_rectangles.head()


df_rectangles shape: (210, 24)


,os_ta_index,geometry,u,v,osmid,name,reversed,length,tunnel,bridge,...,ta_name,overlapping_names,overlapping_osmids,overlapping_names_len,start_edge_idx,end_edge_idx,start_name,end_name,is_connected_to_rab,rectangle
0,6844,"LINESTRING (669191.831 3557280.081, 669191.063...",1750275775,12215833806,1216081179,2433,False,11.024,,,...,2433,,,0,6845,9443,סאמט שמעון,סאמט שמעון,0,"(10901313465, 12215833806, 1750275775, 2373857..."
1,7675,"LINESTRING (669172.362 3557244.025, 669178.433...",2373857201,1750275775,1216081179,2433,False,41.085,,,...,2433,,,0,7674,6844,2433,2433,0,"(10901313465, 12215833806, 1750275775, 2373857..."
2,9349,"LINESTRING (669181.350 3557283.932, 669191.831...",10901313465,1750275775,1216081177,שמעון סאמט,False,11.151,,,...,סאמט שמעון,,,0,9350,6844,2433,2433,0,"(10901313465, 12215833806, 1750275775, 2373857..."
3,9350,"LINESTRING (669181.350 3557283.932, 669180.601...",10901313465,2373857201,1216081178,2433,False,41.024,,,...,2433,,,0,9349,7674,סאמט שמעון,2433,0,"(10901313465, 12215833806, 1750275775, 2373857..."
4,9521,"LINESTRING (669191.063 3557291.047, 669181.350...",12215833806,10901313465,1216081178,2433,False,12.036,,,...,2433,,,0,6844,9349,2433,סאמט שמעון,0,"(10901313465, 12215833806, 1750275775, 2373857..."


In [82]:
# ## checking created polygons

# m = leafmap.Map(center=(32.047, 34.785), zoom=11)


# m.add_gdf(os_ta_streets_nodes_cp)
# m.add_gdf(os_ta_streets_edges)
# m.add_gdf(df_rectangles, fill_colors='black')
# m.add_gdf(df_triangles, fill_colors='black')



# m

Plan:
* create rect polygons
    * add column of rect_idx
    * get area
* Create medium crossroads polygons
    * add cross_idx
    * calculate area
* sjoin between cross_poly and os_ta_streets_edges
    * group by cross_idx
    * get unique os_ta_idx
* sjoin between cross_poly and rect_poly
    * group by rect_idx
    * count how many cross_poly fall in
    * calculate area share

In [83]:
df_rect_cp = df_rectangles.copy()

#### create triangle polygons
df_rect_cp.geometry = df_rect_cp.buffer(6)

df_rect_cp_uni = unary_union(df_rect_cp.geometry)
df_rect_cp_uni = gpd.GeoDataFrame(geometry=[df_rect_cp_uni], crs=df_rect_cp.crs)

## explode
df_rect_cp_explode = df_rect_cp_uni.explode(index_parts=True).reset_index(drop=True)
df_rect_cp_explode.reset_index(inplace=True)

## adding idx column to later be used to filter 
df_rect_cp_explode.columns = ['rect_id','geometry']
df_rect_cp_explode['rect_area'] = df_rect_cp_explode.geometry.area

df_rect_cp_explode['rect_id'] = df_rect_cp_explode['rect_id'].apply(lambda x: str(x) + '_rect_id')
display(df_rect_cp_explode), df_rect_cp_explode.shape


## ALREADY been done
# #### creating cross road poly
# os_ta_streets_edges_cp = os_ta_streets_edges.copy()
# os_ta_streets_edges_split = split_linestrings_by_length(os_ta_streets_edges_cp, 6)
# os_ta_streets_edges_split.geometry = os_ta_streets_edges_split.buffer(6)

# os_ta_streets_edges_uni = unary_union(os_ta_streets_edges_split.geometry)
# os_ta_streets_edges_uni = gpd.GeoDataFrame(geometry=[os_ta_streets_edges_uni], crs=os_ta_streets_edges.crs)
# ## explode
# cross_cp_explode = os_ta_streets_edges_uni.explode(index_parts=True).reset_index(drop=True)
# cross_cp_explode.reset_index(inplace=True)

# ## adding idx column to later be used to filter 
# cross_cp_explode.columns = ['cr_id','geometry']
# cross_cp_explode['cr_area'] = cross_cp_explode.geometry.area

# cross_cp_explode['cr_id'] = cross_cp_explode['cr_id'].apply(lambda x: str(x) + '_cr_id')
# display(cross_cp_explode), cross_cp_explode.shape

,rect_id,geometry,rect_area
0,0_rect_id,"POLYGON ((669776.223 3546761.848, 669776.403 3...",2016.667668
1,1_rect_id,"POLYGON ((666029.118 3547032.445, 666029.049 3...",4444.238925
2,2_rect_id,"POLYGON ((665903.741 3547310.766, 665903.891 3...",688.354852
3,3_rect_id,"POLYGON ((666190.927 3547322.732, 666181.457 3...",1371.932244
4,4_rect_id,"POLYGON ((666659.427 3547529.141, 666659.683 3...",1311.647197
5,5_rect_id,"POLYGON ((665249.349 3547541.264, 665246.772 3...",1729.283377
6,6_rect_id,"POLYGON ((669147.438 3547826.992, 669147.429 3...",1001.909706
7,7_rect_id,"POLYGON ((665705.204 3547961.278, 665705.380 3...",2931.335449
8,8_rect_id,"POLYGON ((669189.239 3548413.487, 669189.267 3...",736.702284
9,9_rect_id,"POLYGON ((667071.772 3548859.085, 667071.798 3...",2401.537096


(None, (34, 3))

#### sjoin between cross_poly and os_ta_streets_edges
* group by cross_idx
* get unique os_ta_idx

In [84]:
# Already been done

# cross_sjoin = gpd.sjoin(cross_cp_explode, os_ta_streets_edges, how='inner', predicate='intersects')
# cross_sjoingb_os_id = cross_sjoin.groupby('cr_id').os_ta_index.unique()
# # cross_sjoingb_os_id
# cross_cp_explode['os_id'] = cross_cp_explode['cr_id'].map(lambda x: cross_sjoingb_os_id[x])
# cross_cp_explode

In [85]:
cr_rect_joined = gpd.sjoin(cross_cp_explode, df_rect_cp_explode, how='inner', predicate='intersects')

cr_rect_gb = cr_rect_joined.groupby('rect_id').agg({
    'cr_id':  lambda g: g.count(),
    'cr_area':  lambda g: g.sum(),
    'os_id': lambda g: list(set(chain.from_iterable(g)))
}).reset_index()


cr_rect_gb.columns = ['rect_id','cr_cnt','cr_area', 'os_id']
df_rect_cp_explode['cr_cnt'] = df_rect_cp_explode['rect_id'].map(lambda x: cr_rect_gb[cr_rect_gb.rect_id == x]['cr_cnt'].values[0])
df_rect_cp_explode['cr_area'] = df_rect_cp_explode['rect_id'].map(lambda x: cr_rect_gb[cr_rect_gb.rect_id == x]['cr_area'].values[0])
df_rect_cp_explode['os_id'] = df_rect_cp_explode['rect_id'].map(lambda x: cr_rect_gb[cr_rect_gb.rect_id == x]['os_id'].values[0])


df_rect_cp_explode

,rect_id,geometry,rect_area,cr_cnt,cr_area,os_id
0,0_rect_id,"POLYGON ((669776.223 3546761.848, 669776.403 3...",2016.667668,2,970.004823,"[9018, 9013, 9008, 9015, 9012, 9019, 9017, 901..."
1,1_rect_id,"POLYGON ((666029.118 3547032.445, 666029.049 3...",4444.238925,4,1269.373828,"[6936, 6947, 6929, 6948, 6953, 6938, 6946, 6937]"
2,2_rect_id,"POLYGON ((665903.741 3547310.766, 665903.891 3...",688.354852,2,872.484133,"[3325, 9151, 9149, 9150, 9148, 7871, 7870, 940..."
3,3_rect_id,"POLYGON ((666190.927 3547322.732, 666181.457 3...",1371.932244,3,1617.739332,"[3695, 3332, 7568, 761, 3331, 7670, 1680, 7566..."
4,4_rect_id,"POLYGON ((666659.427 3547529.141, 666659.683 3...",1311.647197,2,1154.218159,"[5942, 6561, 1698, 3658, 6562, 9069, 5265, 170..."
5,5_rect_id,"POLYGON ((665249.349 3547541.264, 665246.772 3...",1729.283377,3,2084.651726,"[6825, 1765, 1773, 1772, 8391, 6824, 5450, 545..."
6,6_rect_id,"POLYGON ((669147.438 3547826.992, 669147.429 3...",1001.909706,2,1284.908977,"[9267, 4022, 9266, 4023, 513, 4024, 514, 4480,..."
7,7_rect_id,"POLYGON ((665705.204 3547961.278, 665705.380 3...",2931.335449,4,1207.150337,"[4329, 8300, 1782, 4326, 4327, 4325, 4328]"
8,8_rect_id,"POLYGON ((669189.239 3548413.487, 669189.267 3...",736.702284,1,1729.140057,"[6675, 1447, 5904, 7141, 7647, 5905, 7648, 590..."
9,9_rect_id,"POLYGON ((667071.772 3548859.085, 667071.798 3...",2401.537096,2,1285.875633,"[8435, 2064, 8434, 2063, 8414, 8436, 2062, 273..."


In [86]:
df_rect_cp_explode['cr_share'] = df_rect_cp_explode['cr_area'] / df_rect_cp_explode['rect_area']
df_rect_cp_explode['cr_share'] = df_rect_cp_explode['cr_share'].apply(lambda x: round(x, 2))
df_rect_cp_explode

,rect_id,geometry,rect_area,cr_cnt,cr_area,os_id,cr_share
0,0_rect_id,"POLYGON ((669776.223 3546761.848, 669776.403 3...",2016.667668,2,970.004823,"[9018, 9013, 9008, 9015, 9012, 9019, 9017, 901...",0.48
1,1_rect_id,"POLYGON ((666029.118 3547032.445, 666029.049 3...",4444.238925,4,1269.373828,"[6936, 6947, 6929, 6948, 6953, 6938, 6946, 6937]",0.29
2,2_rect_id,"POLYGON ((665903.741 3547310.766, 665903.891 3...",688.354852,2,872.484133,"[3325, 9151, 9149, 9150, 9148, 7871, 7870, 940...",1.27
3,3_rect_id,"POLYGON ((666190.927 3547322.732, 666181.457 3...",1371.932244,3,1617.739332,"[3695, 3332, 7568, 761, 3331, 7670, 1680, 7566...",1.18
4,4_rect_id,"POLYGON ((666659.427 3547529.141, 666659.683 3...",1311.647197,2,1154.218159,"[5942, 6561, 1698, 3658, 6562, 9069, 5265, 170...",0.88
5,5_rect_id,"POLYGON ((665249.349 3547541.264, 665246.772 3...",1729.283377,3,2084.651726,"[6825, 1765, 1773, 1772, 8391, 6824, 5450, 545...",1.21
6,6_rect_id,"POLYGON ((669147.438 3547826.992, 669147.429 3...",1001.909706,2,1284.908977,"[9267, 4022, 9266, 4023, 513, 4024, 514, 4480,...",1.28
7,7_rect_id,"POLYGON ((665705.204 3547961.278, 665705.380 3...",2931.335449,4,1207.150337,"[4329, 8300, 1782, 4326, 4327, 4325, 4328]",0.41
8,8_rect_id,"POLYGON ((669189.239 3548413.487, 669189.267 3...",736.702284,1,1729.140057,"[6675, 1447, 5904, 7141, 7647, 5905, 7648, 590...",2.35
9,9_rect_id,"POLYGON ((667071.772 3548859.085, 667071.798 3...",2401.537096,2,1285.875633,"[8435, 2064, 8434, 2063, 8414, 8436, 2062, 273...",0.54


Checking different share to determine what os_id to take and then drop from df_triangle

In [87]:
print('shape of 0.25 below share',df_rect_cp_explode[df_rect_cp_explode.cr_share < 0.25].shape)
print('shape of 0.35 below share',df_rect_cp_explode[df_rect_cp_explode.cr_share < 0.35].shape)
print('shape of 0.45 below share',df_rect_cp_explode[df_rect_cp_explode.cr_share < 0.45].shape)
print('shape of 0.55 below share',df_rect_cp_explode[df_rect_cp_explode.cr_share < 0.55].shape)
print('shape of 0.65 below share',df_rect_cp_explode[df_rect_cp_explode.cr_share < 0.65].shape)




shape of 0.25 below share (2, 7)
shape of 0.35 below share (3, 7)
shape of 0.45 below share (5, 7)
shape of 0.55 below share (9, 7)
shape of 0.65 below share (9, 7)


In [88]:
df_rect_cp_explode[(df_rect_cp_explode.cr_share < 1.15) & (df_rect_cp_explode.cr_share >= 1.05)].shape

(1, 7)

In [89]:
# ## checking created polygons

# m = leafmap.Map(center=(32.047, 34.785), zoom=11)


# m.add_gdf(df_rect_cp_explode)

# m

In [90]:
rect_id_to_keep  =[]

rect_to_maybe_keep = []

rect_id_to_drop = ['1_rect_id', '7_rect_id','9_rect_id','14_rect_id','18_rect_id','27_rect_id']

Getting os_id from list of drop and keep and preforming accordingly

In [91]:
os_id_drop_ls = df_rect_cp_explode[df_rect_cp_explode.rect_id.isin(rect_id_to_drop)].os_id.to_list()
os_id_drop_ls = list(set(chain.from_iterable(os_id_drop_ls)))
os_id_drop_ls
print('os_id_drop_ls',len(os_id_drop_ls))

os_id_drop_ls 65


Making sure what I keep and what I drop make sense

In [92]:
# ## checking created polygons

# m = leafmap.Map(center=(32.047, 34.785), zoom=11)


# m.add_gdf(df_rectangles[df_rectangles.os_ta_index.isin(os_id_drop_ls)])

# m

Dropping from triangle

There is a difference in number of os_id in drop list compared to what is in df_triangles<br>
This is because when creating crossroads we caught edges that weren't in def_triangles. 

In [93]:
df_rectangles_clean = df_rectangles[~df_rectangles.os_ta_index.isin(os_id_drop_ls)]


In [94]:
df_rectangles_clean = df_rectangles_clean[(df_rectangles_clean.junction != 'roundabout') & (df_rectangles_clean.is_connected_to_rab == 0)]

In [95]:
df_rectangles_clean.to_parquet('./csv_tables/edges_rectangles.parquet')

### Exporting roundabout data
We already have this so we can simply export

In [96]:
rab_to_fix_idx = os_ta_streets_edges[(os_ta_streets_edges.junction == 'roundabout') & (os_ta_streets_edges.is_connected_to_rab == 1)].index
os_ta_streets_edges.loc[rab_to_fix_idx, 'is_connected_to_rab'] = 0

In [97]:
os_ta_streets_edges.to_parquet('./csv_tables/os_ta_streets_edges.parquet')

In [98]:
ra_cr = os_ta_streets_edges[(os_ta_streets_edges.junction == 'roundabout')].copy()
print(ra_cr.shape)
ra_cr.head(3)

(515, 23)


,os_ta_index,geometry,u,v,osmid,name,reversed,length,tunnel,bridge,...,name_fixed,ta_name,overlapping_names,overlapping_osmids,overlapping_names_len,start_edge_idx,end_edge_idx,start_name,end_name,is_connected_to_rab
177,177,"LINESTRING (668874.869 3551329.236, 668885.399...",35288627,2203627170,5118376,ה' באייר,False,27.759999999999998,,,...,ה' באייר,ה' באייר,"הא באייר ,ז'בוטינסקי ,ויצמן ,חברה חדשה ,תש״ח ,...",5118376,6,178,1284,ויצמן,ויצמן,0
256,256,"LINESTRING (672928.438 3554590.534, 672933.715...",271878297,381571872,33492500,כיכר ומשמר הירדן,False,28.725,,,...,,כיכר ומשמר הירדן,",משמר הירדן ,מרכוס דוד ,פתחיה מרגנשבורג","1134738136 ,33492500",4,257,1900,משמר הירדן,מרכוס דוד,0
258,258,"LINESTRING (672940.500 3554645.491, 672933.864...",271878300,1170469323,33492500,כיכר ומשמר הירדן,False,15.693,,,...,,כיכר ומשמר הירדן,",משמר הירדן ,מרכוס דוד ,פתחיה מרגנשבורג","1134738136 ,33492500",4,251,2745,משמר הירדן,פתחיה מרגנשבורג,0


In [99]:
ra_cr.to_parquet('./csv_tables/edges_roundabout.parquet')

In [100]:
# ## checking created polygons

# m = leafmap.Map(center=(32.047, 34.785), zoom=11)


# # m.add_gdf(ra_cr)
# m.add_gdf( os_ta_streets_edges[(os_ta_streets_edges.is_connected_to_rab == 1)], fill_colors='black')
# m.add_gdf(ra_cr)



# m

### Looking at Roundabout Triangle and rectangle Crossroads

In [101]:
rect_edges = gpd.read_parquet('./csv_tables/edges_rectangles.parquet')
tri_edges  = gpd.read_parquet('./csv_tables/edges_triangles.parquet')
ra_edges   = gpd.read_parquet('./csv_tables/edges_roundabout.parquet')


In [102]:
# ## checking created polygons

# m = leafmap.Map(center=(32.047, 34.785), zoom=11)


# # m.add_gdf(ra_cr)
# m.add_gdf(tri_cr)
# m.add_gdf(rect_cr, fill_colors='black')
# # m.add_gdf(ra_cr)

# m

### Dropping edges that exist in tri and rect and exporting again

In [103]:
rect_os_ta_index = rect_edges.os_ta_index.to_list()
os_ta_index_to_filter = tri_edges[tri_edges.os_ta_index.isin(rect_os_ta_index)].os_ta_index

rect_edges = rect_edges[~rect_edges.os_ta_index.isin(os_ta_index_to_filter)].copy()

rect_edges.to_parquet('./csv_tables/edges_rectangles.parquet')

In [104]:
# ## checking created polygons

# m = leafmap.Map(center=(32.047, 34.785), zoom=11)
# m.add_gdf(tri_edges[tri_edges.os_ta_index.isin(rect_os_ta_index)], fill_colors='black')

# m

###  Creating edges that not in rect, tri, and ra

In [105]:
rect_edges = gpd.read_parquet('./csv_tables/edges_rectangles.parquet')
tri_edges  = gpd.read_parquet('./csv_tables/edges_triangles.parquet')
ra_edges   = gpd.read_parquet('./csv_tables/edges_roundabout.parquet')


In [106]:
os_ta_streets_edges[(~os_ta_streets_edges.os_ta_index.isin(rect_edges.os_ta_index)) & 
                    (~os_ta_streets_edges.os_ta_index.isin(tri_edges.os_ta_index)) & 
                    (~os_ta_streets_edges.os_ta_index.isin(ra_edges.os_ta_index))].to_parquet('./csv_tables/edges_rest.parquet')

## Creating polygons from tri, rect, ra and rest

In [107]:
rect_edges = gpd.read_parquet('./csv_tables/edges_rectangles.parquet')
tri_edges  = gpd.read_parquet('./csv_tables/edges_triangles.parquet')
ra_edges   = gpd.read_parquet('./csv_tables/edges_roundabout.parquet')
rest_edges = gpd.read_parquet('./csv_tables/edges_rest.parquet')

### rect polygon

In [108]:
# Buffer
rect_edges_cp = rect_edges.copy()
rect_edges_cp.geometry = rect_edges_cp.buffer(5)

rect_edges_uni =  unary_union(rect_edges_cp.geometry)

# gdf
rect_edges_uni_gdf = gpd.GeoDataFrame(geometry=[rect_edges_uni], crs=rect_edges_cp.crs)
rect_edges_poly_gdf = rect_edges_uni_gdf.explode(index_parts=True).reset_index(drop=True)
rect_edges_poly_gdf.geometry = rect_edges_poly_gdf.convex_hull

### tri polygon

In [109]:
# Buffer
tri_edges_cp = tri_edges.copy()
tri_edges_cp.geometry = tri_edges_cp.buffer(5)

tri_edges_uni =  unary_union(tri_edges_cp.geometry)

# gdf
tri_edges_uni_gdf = gpd.GeoDataFrame(geometry=[tri_edges_uni], crs=tri_edges_cp.crs)
tri_edges_poly_gdf = tri_edges_uni_gdf.explode(index_parts=True).reset_index(drop=True)
tri_edges_poly_gdf
tri_edges_poly_gdf.geometry = tri_edges_poly_gdf.convex_hull

### ra polygon

In [110]:
# Buffer
ra_edges_cp = ra_edges.copy()
ra_edges_cp.geometry = ra_edges_cp.buffer(5)

ra_edges_uni =  unary_union(ra_edges_cp.geometry)

# gdf
ra_edges_uni_gdf = gpd.GeoDataFrame(geometry=[ra_edges_uni], crs=ra_edges_cp.crs)
ra_edges_poly_gdf = ra_edges_uni_gdf.explode(index_parts=True).reset_index(drop=True)
ra_edges_poly_gdf.geometry = ra_edges_poly_gdf.convex_hull

In [111]:
# ## checking created polygons

# m = leafmap.Map(center=(32.047, 34.785), zoom=11)
# m.add_gdf(tri_edges_poly_gdf)
# # m.add_gdf(tri_edges,fill_colors='black')


# m

## Creating Crossroads and checking how they look

* create crossroads using edges_rest with:
    * rect_edges
    * tri_edges 
    * ra_edges
    * view on map

plan:
* split edges_rest
* buffer edges_rest
* union with buffed rect/tri/ra/edges
* explode to single polygons
* view on map

If good:
* sjoin the result on os_ta_edges
* name each crossroad
* export 


In [112]:
rect_edges = gpd.read_parquet('./csv_tables/edges_rectangles.parquet')
tri_edges  = gpd.read_parquet('./csv_tables/edges_triangles.parquet')
ra_edges   = gpd.read_parquet('./csv_tables/edges_roundabout.parquet')
rest_edges = gpd.read_parquet('./csv_tables/edges_rest.parquet')

In [113]:
# split edges_rest
rest_edges_split_ends, rest_edges_split_between = split_linestrings_by_length_return_two_gdfs( rest_edges, 6)
rest_edges_split_ends.shape

# buffer edges_rest
rest_edges_split_ends.geometry = rest_edges_split_ends.buffer(5.5, cap_style='square')
rest_edges_split_ends


# union with rect/tri/ra
all_geoms_ls = rest_edges_split_ends.geometry.to_list() + rect_edges_poly_gdf.geometry.to_list() + tri_edges_poly_gdf.geometry.to_list() + ra_edges_poly_gdf.geometry.to_list()
all_geoms = gpd.GeoSeries(all_geoms_ls)
# # len(rest_and_rect_geoms)
all_geoms_uni = unary_union(all_geoms)
all_geoms_gdf = gpd.GeoDataFrame(geometry=[all_geoms_uni], crs=rest_edges.crs)
all_geoms_poly_gdf = all_geoms_gdf.explode(index_parts=True).reset_index(drop=True)

In [114]:
# ## checking created polygons

# m = leafmap.Map(center=(32.047, 34.785), zoom=11)
# m.add_gdf(all_geoms_poly_gdf)

# m

In [115]:
rest_edges_split_between.to_parquet('./csv_tables/edges_rest_between_cr.parquet')

In [116]:
all_geoms_poly_gdf.to_parquet('./csv_tables/ta_poly_crossroads.parquet')

### conclusion of creating geoms:

I think I have reached a point where trying to get better results is not worth the efforts. 

What are some of the issues **all_geoms_poly_gdf** currently has:
* some streets in florentin are connected while they shouldn't<br>
  This is because of how small and short these streets are.
* some area like **montifiory** are not connected well even though they should

There are more places with this kind of issue.

My next task is:
* Add area to crossroads
* Add an identifier
* combine the data from os_ta_streets_edges with the all_geoms_poly_gdf.
* Give each polygon/crossroad a name
* add number of edges each polygon is constructed
* Add which edges it is constructed
* Add which nodes it is constructed


In [117]:
all_geoms_poly_gdf = gpd.read_parquet('./csv_tables/ta_poly_crossroads.parquet')

#### Add area to crossroads

In [118]:
all_geoms_poly_gdf['cr_area'] = all_geoms_poly_gdf.area

#### Give each polygon/crossroad a name and which edges it is constructed

In [119]:
all_geoms_poly_gdf.reset_index(inplace=True)

In [120]:
all_geoms_poly_gdf.columns

Index(['index', 'geometry', 'cr_area'], dtype='object')

In [121]:
all_geoms_poly_gdf.columns = ['cr_idx','geometry','cr_area']

Connect os_ta_edges with all_geoms_poly_gdf

In [122]:
ta_edges_all_cr_joined = gpd.sjoin(all_geoms_poly_gdf, os_ta_streets_edges, how='inner', predicate='intersects')

ta_edges_all_cr_joined.shape

(16220, 26)

In [123]:
def get_cr_name(arr):
    # This list will hold all the individual items after splitting
    new_items = []

    for s in arr:
        # Split by comma to separate values
        parts = s.split(',')
        for part in parts:
            # Strip spaces and commas
            stripped_part = part.strip(" ,")
            # Only add if not empty string
            if stripped_part:
                new_items.append(stripped_part)

    # Remove duplicates using set, then convert back to list
    unique_items = list(set(new_items))
    
    # Sort alphabetically (optional, if desired)
    unique_items.sort()

    return unique_items


In [124]:
ta_joined_gb_ta_name = ta_edges_all_cr_joined.groupby('cr_idx').agg({'ta_name':get_cr_name, 'os_ta_index':pd.unique}) 

ta_joined_gb_ta_name.columns = ['ta_name_ls', 'os_ta_index']
ta_joined_gb_ta_name['ta_name'] = ta_joined_gb_ta_name.ta_name_ls.apply(lambda x:  ' ,'.join(x))
ta_joined_gb_ta_name

,ta_name_ls,os_ta_index,ta_name
cr_idx,,,
0,"[הגבול (שמחה הולצברג), שדרות ירושלים]","[695, 9224]","הגבול (שמחה הולצברג) ,שדרות ירושלים"
1,[הגבול (שמחה הולצברג)],"[695, 659]",הגבול (שמחה הולצברג)
2,"[הגבול (שמחה הולצברג), קרוא ברוך]","[659, 660, 661]","הגבול (שמחה הולצברג) ,קרוא ברוך"
3,[מייזל זלמן],[668],מייזל זלמן
4,"[מייזל זלמן, קרוא ברוך]","[660, 668, 669, 667]","מייזל זלמן ,קרוא ברוך"
...,...,...,...
4259,"[2380, זלצמן פנינה פרופ']","[7934, 8960, 7936, 7935, 8959]","2380 ,זלצמן פנינה פרופ'"
4260,"[2170, יוניצ'מן, כיכר ו2170]","[1861, 9437, 1862, 5173]","2170 ,יוניצ'מן ,כיכר ו2170"
4261,[דרך תל-אביב],"[5274, 5273]",דרך תל-אביב


In [125]:
ta_edges_all_cr_joined['cr_name'] = ta_edges_all_cr_joined.cr_idx.map(lambda x: ta_joined_gb_ta_name.loc[x, 'ta_name'])
ta_edges_all_cr_joined['cr_name_ls'] = ta_edges_all_cr_joined.cr_idx.map(lambda x: ta_joined_gb_ta_name.loc[x, 'ta_name_ls'])
ta_edges_all_cr_joined['cr_edges'] = ta_edges_all_cr_joined.cr_idx.map(lambda x: ta_joined_gb_ta_name.loc[x, 'os_ta_index'])


#### Adding which nodes are related to crossroads 

In [126]:
ta_nodes_all_cr_joined = gpd.sjoin(os_ta_streets_nodes, all_geoms_poly_gdf, how='inner', predicate='intersects')

ta_nodes_all_cr_joined.head(3)

,osmid,y,x,highway,street_count,ref,geometry,is_roundabout,index_right,cr_idx,cr_area
0,139693,32.093840,34.790572,traffic_signals,4,None,POINT (668968.683 3552240.237),0,3195,3195,723.636262
1,139698,32.093869,34.791231,None,3,None,POINT (669030.815 3552244.552),0,3197,3197,319.573148
2,139707,32.095354,34.778500,None,3,None,POINT (667826.578 3552389.242),0,3242,3242,321.319308


In [127]:
nodes_cr_gb = ta_nodes_all_cr_joined.groupby('cr_idx').osmid.unique()
nodes_cr_gb

cr_idx
0                                            [318180661]
1                                            [318177898]
2                                            [318177899]
3                                            [318178557]
4                                            [318178468]
                              ...                       
4259    [2868373867, 6822274805, 6822274807, 6822274808]
4260                            [374352685, 11267492233]
4261                                        [1144043820]
4262                                        [1131244786]
4263                                        [9931532351]
Name: osmid, Length: 4264, dtype: object

#### Adding to ta_edges_all_cr_joined the nodes related to cr

In [128]:
ta_edges_all_cr_joined['cr_nodes'] = ta_edges_all_cr_joined.cr_idx.map(lambda x: nodes_cr_gb[x])

In [129]:
ta_edges_all_cr_joined.loc[0, 'cr_name']

0    הגבול (שמחה הולצברג) ,שדרות ירושלים
0    הגבול (שמחה הולצברג) ,שדרות ירושלים
Name: cr_name, dtype: object

In [130]:
ta_edges_all_cr_joined.head(3)

,cr_idx,geometry,cr_area,index_right,os_ta_index,u,v,osmid,name,reversed,...,overlapping_names_len,start_edge_idx,end_edge_idx,start_name,end_name,is_connected_to_rab,cr_name,cr_name_ls,cr_edges,cr_nodes
0,0,"POLYGON ((665228.049 3545035.251, 665223.517 3...",253.932777,695,695,318180661,318177898,1124797596,הגבול (שמחה הולצברג),False,...,0,9224,659,שדרות ירושלים,הגבול (שמחה הולצברג),0,"הגבול (שמחה הולצברג) ,שדרות ירושלים","[הגבול (שמחה הולצברג), שדרות ירושלים]","[695, 9224]",[318180661]
0,0,"POLYGON ((665228.049 3545035.251, 665223.517 3...",253.932777,9218,9224,9971016122,318180661,1124797593,שדרות ירושלים,False,...,0,693,695,שדרות ירושלים,הגבול (שמחה הולצברג),0,"הגבול (שמחה הולצברג) ,שדרות ירושלים","[הגבול (שמחה הולצברג), שדרות ירושלים]","[695, 9224]",[318180661]
1,1,"POLYGON ((665172.569 3545071.176, 665167.936 3...",253.065302,695,695,318180661,318177898,1124797596,הגבול (שמחה הולצברג),False,...,0,9224,659,שדרות ירושלים,הגבול (שמחה הולצברג),0,הגבול (שמחה הולצברג),[הגבול (שמחה הולצברג)],"[695, 659]",[318177898]


In [131]:
ta_edges_all_cr_joined.columns

Index(['cr_idx', 'geometry', 'cr_area', 'index_right', 'os_ta_index', 'u', 'v',
       'osmid', 'name', 'reversed', 'length', 'tunnel', 'bridge', 'junction',
       'name_type', 'osmid_type', 'name_fixed', 'ta_name', 'overlapping_names',
       'overlapping_osmids', 'overlapping_names_len', 'start_edge_idx',
       'end_edge_idx', 'start_name', 'end_name', 'is_connected_to_rab',
       'cr_name', 'cr_name_ls', 'cr_edges', 'cr_nodes'],
      dtype='object')

In [132]:
ta_edges_all_cr_joined['cr_edges_cnt'] = ta_edges_all_cr_joined['cr_edges'].apply(lambda x: len(x))
ta_edges_all_cr_joined['cr_nodes_cnt'] = ta_edges_all_cr_joined['cr_nodes'].apply(lambda x: len(x))

In [133]:
cols_to_keep = ['geometry','cr_idx','cr_name_ls', 'cr_name','cr_edges','cr_edges_cnt','cr_nodes','cr_nodes_cnt']

In [134]:
ta_edges_all_cr_joined_no_dup = ta_edges_all_cr_joined.drop_duplicates(subset=['cr_idx','cr_name'])
ta_edges_all_cr_joined_no_dup[cols_to_keep]

,geometry,cr_idx,cr_name_ls,cr_name,cr_edges,cr_edges_cnt,cr_nodes,cr_nodes_cnt
0,"POLYGON ((665228.049 3545035.251, 665223.517 3...",0,"[הגבול (שמחה הולצברג), שדרות ירושלים]","הגבול (שמחה הולצברג) ,שדרות ירושלים","[695, 9224]",2,[318180661],1
1,"POLYGON ((665172.569 3545071.176, 665167.936 3...",1,[הגבול (שמחה הולצברג)],הגבול (שמחה הולצברג),"[695, 659]",2,[318177898],1
2,"POLYGON ((665124.682 3545101.841, 665124.555 3...",2,"[הגבול (שמחה הולצברג), קרוא ברוך]","הגבול (שמחה הולצברג) ,קרוא ברוך","[659, 660, 661]",3,[318177899],1
3,"POLYGON ((665224.294 3545131.637, 665228.846 3...",3,[מייזל זלמן],מייזל זלמן,[668],1,[318178557],1
4,"POLYGON ((665174.747 3545165.316, 665174.827 3...",4,"[מייזל זלמן, קרוא ברוך]","מייזל זלמן ,קרוא ברוך","[660, 668, 669, 667]",4,[318178468],1
...,...,...,...,...,...,...,...,...
4259,"POLYGON ((669581.910 3557713.428, 669577.147 3...",4259,"[2380, זלצמן פנינה פרופ']","2380 ,זלצמן פנינה פרופ'","[7934, 8960, 7936, 7935, 8959]",5,"[2868373867, 6822274805, 6822274807, 6822274808]",4
4260,"POLYGON ((669263.552 3557726.504, 669268.737 3...",4260,"[2170, יוניצ'מן, כיכר ו2170]","2170 ,יוניצ'מן ,כיכר ו2170","[1861, 9437, 1862, 5173]",4,"[374352685, 11267492233]",2
4261,"POLYGON ((669963.496 3557730.701, 669962.242 3...",4261,[דרך תל-אביב],דרך תל-אביב,"[5274, 5273]",2,[1144043820],1
4262,"POLYGON ((669028.902 3557743.422, 669030.676 3...",4262,[2170],2170,[5173],1,[1131244786],1


In [135]:
ta_edges_all_cr_joined_no_dup[ta_edges_all_cr_joined_no_dup.cr_edges_cnt > 1][cols_to_keep].to_parquet('./csv_tables/ta_crossroads_22012025.parquet')

In [136]:
ta_crossroads = gpd.read_parquet('./csv_tables/ta_crossroads_22012025.parquet')

In [137]:
# ## checking created polygons

# m = leafmap.Map(center=(32.047, 34.785), zoom=11)
# m.add_gdf(ta_crossroads)

# m

### Handling specific crossroads and streets segments: The solution was done and is commented out. 


In [138]:
cr_to_combine = [
                (1913,1919,1943), (1760, 1785, 1788, 1805), (2061,2076, 2073),(1996,2004,2018), (1852,1836),
                (2088, 2103), (1827,1814), (2422, 2427, 2444), (2581,2606, 2602), (2801,2811,2810,2818), (2833, 2835,2823, 2839),
                (3023,3039), (3093,3106,3105 ), (3256,3258, ), (3344,3352 ), (3374,3371, 3378 ), (3392, 3395), (3420,3426),
                (4095,4085, 4113), (4007,4020), (4197, 4199), (4249,4250, 4252,4251 ),
                  (3363, 3358),
                  (80, 93,),
                  (98, 96),
                 (321, 288),(498,495, 548,519, 549 ), (625, 690,709 ), (702, 675, 695), (756,758, 776, 787), (1623,1641 ), (817, 809, 853), 
                  (468, 480), (1473, 1484, 1483), (802, 777), (584,558), (584,558,) ,(848, 844)
                  ]

# sg_to_drop = [8266,10669, 9468, 10000,10006,8643, 10606, 12588, 11542, 4149, 4062,10197, 51, 10472,11142, 3062,3060, 3059,
#               7723,11797, 10232,9837,9834,9836,8892,10777, 10589, 8823,11660, 1264,7141,12478, 12225, 6596, 7827, 12268, 918,
#               10841,6917,9634,783, 9648, 65, 8414,12363, 12362,9706, 9697, 241, 424, 10259, 5619,4641, 10026,10027, 10343, 12161,10264,
#               10252, 5561, 5562, 9125, 9768, 9767, 11468  ]

In [139]:
# ## checking created polygons

# m = leafmap.Map(center=(32.047, 34.785), zoom=11)
# m.add_gdf(ta_crossroads[ta_crossroads.cr_idx.isin([1913,1919 ,1943])])

# m

In [140]:
# List for storing the merged geometries
merged_geometries = []
# And possibly a key for identifying each group if you want
group_ids = []

for i, tuple_group in enumerate(cr_to_combine):
    # 1) Subset using 'isin' on your ID column
    subset = ta_crossroads[ta_crossroads['cr_idx'].isin(tuple_group)]
    
    # 2) Perform union of all geometries in the subset
    merged_poly = subset.unary_union  # returns a single Polygon or MultiPolygon
    merged_poly = merged_poly.convex_hull
    
    # Store results
    merged_geometries.append(merged_poly)
    group_ids.append(i)

# 3) Create a new GeoDataFrame holding the merged polygons
merged_gdf = gpd.GeoDataFrame(
    {"cr_idx": group_ids, "geometry": merged_geometries}, 
    crs=ta_crossroads.crs
)

# Now merged_gdf has one row per tuple in cr_to_combine, each row a merged geometry
merged_gdf

merged_gdf['cr_area'] = merged_gdf.area
#### Give each polygon/crossroad a name and which edges it is constructed
merged_gdf.columns
# merged_gdf.columns = ['cr_idx','geometry','cr_area']


Index(['cr_idx', 'geometry', 'cr_area'], dtype='object')

In [141]:
# ## checking created polygons

# m = leafmap.Map(center=(32.047, 34.785), zoom=11)
# m.add_gdf(merged_gdf)

# m

In [142]:
# 1) Flatten all IDs in cr_to_combine into a single list
ids_to_remove = [cr_id for group_tuple in cr_to_combine for cr_id in group_tuple]

# 2) Exclude those IDs from ta_crossroads
ta_crossroads = ta_crossroads[~ta_crossroads["cr_idx"].isin(ids_to_remove)]
ta_crossroads.to_parquet('./csv_tables/ta_crossroads_22012025.parquet')

### Preforming naming and other stuff to handled crossroads

In [143]:
ta_edges_all_cr_joined = gpd.sjoin(merged_gdf, os_ta_streets_edges, how='inner', predicate='intersects')

ta_edges_all_cr_joined.head(3)

ta_joined_gb_ta_name = ta_edges_all_cr_joined.groupby('cr_idx').agg({'ta_name':get_cr_name, 'os_ta_index':pd.unique}) 

ta_joined_gb_ta_name.columns = ['ta_name_ls', 'os_ta_index']
ta_joined_gb_ta_name['ta_name'] = ta_joined_gb_ta_name.ta_name_ls.apply(lambda x:  ' ,'.join(x))


ta_edges_all_cr_joined['cr_name'] = ta_edges_all_cr_joined.cr_idx.map(lambda x: ta_joined_gb_ta_name.loc[x, 'ta_name'])
ta_edges_all_cr_joined['cr_name_ls'] = ta_edges_all_cr_joined.cr_idx.map(lambda x: ta_joined_gb_ta_name.loc[x, 'ta_name_ls'])
ta_edges_all_cr_joined['cr_edges'] = ta_edges_all_cr_joined.cr_idx.map(lambda x: ta_joined_gb_ta_name.loc[x, 'os_ta_index'])


ta_nodes_all_cr_joined = gpd.sjoin(os_ta_streets_nodes, merged_gdf, how='inner', predicate='intersects')

ta_nodes_all_cr_joined.head(3)

nodes_cr_gb = ta_nodes_all_cr_joined.groupby('cr_idx').osmid.unique()

ta_edges_all_cr_joined['cr_nodes'] = ta_edges_all_cr_joined.cr_idx.map(lambda x: nodes_cr_gb[x])


ta_edges_all_cr_joined['cr_edges_cnt'] = ta_edges_all_cr_joined['cr_edges'].apply(lambda x: len(x))
ta_edges_all_cr_joined['cr_nodes_cnt'] = ta_edges_all_cr_joined['cr_nodes'].apply(lambda x: len(x))

cols_to_keep = ['geometry','cr_idx','cr_name_ls', 'cr_name','cr_edges','cr_edges_cnt','cr_nodes','cr_nodes_cnt']

ta_edges_all_cr_joined_no_dup = ta_edges_all_cr_joined.drop_duplicates(subset=['cr_idx','cr_name'])

ta_edges_all_cr_joined_no_dup[ta_edges_all_cr_joined_no_dup.cr_edges_cnt > 1][cols_to_keep].to_parquet('./csv_tables/ta_crossroads_22012025_additional.parquet')

#### Adding new polygons to ta_crossroads

* First remove cr_idx from both
* concat ta_edges_all_cr_joined_no_dup to ta_crossroads
* add cr_idx

In [144]:
ta_crossroads

,geometry,cr_idx,cr_name_ls,cr_name,cr_edges,cr_edges_cnt,cr_nodes,cr_nodes_cnt
0,"POLYGON ((665228.049 3545035.251, 665223.517 3...",0,"[הגבול (שמחה הולצברג), שדרות ירושלים]","הגבול (שמחה הולצברג) ,שדרות ירושלים","[695, 9224]",2,[318180661],1
1,"POLYGON ((665172.569 3545071.176, 665167.936 3...",1,[הגבול (שמחה הולצברג)],הגבול (שמחה הולצברג),"[695, 659]",2,[318177898],1
2,"POLYGON ((665124.682 3545101.841, 665124.555 3...",2,"[הגבול (שמחה הולצברג), קרוא ברוך]","הגבול (שמחה הולצברג) ,קרוא ברוך","[659, 660, 661]",3,[318177899],1
4,"POLYGON ((665174.747 3545165.316, 665174.827 3...",4,"[מייזל זלמן, קרוא ברוך]","מייזל זלמן ,קרוא ברוך","[660, 668, 669, 667]",4,[318178468],1
5,"POLYGON ((665321.973 3545160.286, 665319.071 3...",5,[שדרות ירושלים],שדרות ירושלים,"[9224, 693]",2,[9971016122],1
...,...,...,...,...,...,...,...,...
4258,"POLYGON ((669624.865 3557684.649, 669630.083 3...",4258,[זלצמן פנינה פרופ'],זלצמן פנינה פרופ',"[7933, 7934, 8960]",3,[2868373857],1
4259,"POLYGON ((669581.910 3557713.428, 669577.147 3...",4259,"[2380, זלצמן פנינה פרופ']","2380 ,זלצמן פנינה פרופ'","[7934, 8960, 7936, 7935, 8959]",5,"[2868373867, 6822274805, 6822274807, 6822274808]",4
4260,"POLYGON ((669263.552 3557726.504, 669268.737 3...",4260,"[2170, יוניצ'מן, כיכר ו2170]","2170 ,יוניצ'מן ,כיכר ו2170","[1861, 9437, 1862, 5173]",4,"[374352685, 11267492233]",2
4261,"POLYGON ((669963.496 3557730.701, 669962.242 3...",4261,[דרך תל-אביב],דרך תל-אביב,"[5274, 5273]",2,[1144043820],1


In [145]:
ta_crossroads = gpd.read_parquet('./csv_tables/ta_crossroads_22012025.parquet')
ta_crossroads_additional = gpd.read_parquet('./csv_tables/ta_crossroads_22012025_additional.parquet')

In [146]:
ta_crossroads_additional.columns, ta_crossroads.columns

(Index(['geometry', 'cr_idx', 'cr_name_ls', 'cr_name', 'cr_edges',
        'cr_edges_cnt', 'cr_nodes', 'cr_nodes_cnt'],
       dtype='object'),
 Index(['geometry', 'cr_idx', 'cr_name_ls', 'cr_name', 'cr_edges',
        'cr_edges_cnt', 'cr_nodes', 'cr_nodes_cnt'],
       dtype='object'))

In [147]:
ta_crossroads.drop(columns=['cr_idx'],inplace=True)
ta_crossroads_additional.drop(columns=['cr_idx'], inplace=True)

df_combined = pd.concat([ta_crossroads, ta_crossroads_additional], axis=0, ignore_index=True)

gdf_combined = gpd.GeoDataFrame(df_combined, geometry='geometry', crs=ta_crossroads.crs)
gdf_combined = gdf_combined.reset_index()
cols = gdf_combined.columns
cols

Index(['index', 'geometry', 'cr_name_ls', 'cr_name', 'cr_edges',
       'cr_edges_cnt', 'cr_nodes', 'cr_nodes_cnt'],
      dtype='object')

In [148]:
gdf_combined.columns = ['cr_idx', 'geometry', 'cr_name_ls', 'cr_name', 'cr_edges',
       'cr_edges_cnt', 'cr_nodes', 'cr_nodes_cnt']

In [149]:
gdf_combined.to_parquet('./csv_tables/ta_crossroads_22012025.parquet')

In [150]:
## checking created polygons

m = leafmap.Map(center=(32.047, 34.785), zoom=11)
m.add_gdf(gdf_combined,fill_colors='black')
# m.add_gdf(ta_streets_no_cr)


m

Map(center=[32.047, 34.785], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom…

### Creating street segments that exclude crossroads

We buffer the ta_crossroads again since after several attempts we found that accidents on the street are missing.

In [151]:
ta_crossroads = gpd.read_parquet('./csv_tables/ta_crossroads_22012025.parquet')

In [152]:
ta_crossroads

,cr_idx,geometry,cr_name_ls,cr_name,cr_edges,cr_edges_cnt,cr_nodes,cr_nodes_cnt
0,0,"POLYGON ((665228.049 3545035.251, 665223.517 3...","[הגבול (שמחה הולצברג), שדרות ירושלים]","הגבול (שמחה הולצברג) ,שדרות ירושלים","[695, 9224]",2,[318180661],1
1,1,"POLYGON ((665172.569 3545071.176, 665167.936 3...",[הגבול (שמחה הולצברג)],הגבול (שמחה הולצברג),"[695, 659]",2,[318177898],1
2,2,"POLYGON ((665124.682 3545101.841, 665124.555 3...","[הגבול (שמחה הולצברג), קרוא ברוך]","הגבול (שמחה הולצברג) ,קרוא ברוך","[659, 660, 661]",3,[318177899],1
3,3,"POLYGON ((665174.747 3545165.316, 665174.827 3...","[מייזל זלמן, קרוא ברוך]","מייזל זלמן ,קרוא ברוך","[660, 668, 669, 667]",4,[318178468],1
4,4,"POLYGON ((665321.973 3545160.286, 665319.071 3...",[שדרות ירושלים],שדרות ירושלים,"[9224, 693]",2,[9971016122],1
...,...,...,...,...,...,...,...,...
3522,3522,"POLYGON ((665644.994 3547906.799, 665610.317 3...","[העליה השניה, רוסלאן, רציף העלייה השנייה]","העליה השניה ,רוסלאן ,רציף העלייה השנייה","[4327, 1782, 4325, 4328, 4326, 4329]",6,"[580725909, 580726367, 580726702]",3
3523,3523,"POLYGON ((667655.748 3547130.788, 667644.463 3...","[אסירי ציון, מסילת ישרים, קבוץ גלויות]","אסירי ציון ,מסילת ישרים ,קבוץ גלויות","[6110, 8536, 3915, 7118, 6109, 6139, 7120, 613...",17,"[1566055007, 1566055928, 1566056268, 195048759...",8
3524,3524,"POLYGON ((666437.519 3546857.423, 666426.531 3...","[היינה היינריך, חבר הלאומים, שרירא גאון]","היינה היינריך ,חבר הלאומים ,שרירא גאון","[8323, 9105, 9106, 3710, 7599, 3709, 7600, 8299]",8,"[540440145, 2266143436, 3909311113, 8717879047]",4
3525,3525,"POLYGON ((666437.519 3546857.423, 666426.531 3...","[היינה היינריך, חבר הלאומים, שרירא גאון]","היינה היינריך ,חבר הלאומים ,שרירא גאון","[8323, 9105, 9106, 3710, 7599, 3709, 7600, 8299]",8,"[540440145, 2266143436, 3909311113, 8717879047]",4


In [153]:
os_ta_streets_edges_cp = os_ta_streets_edges.copy()
os_ta_streets_edges_cp.geometry = os_ta_streets_edges_cp.buffer(6.5, cap_style='square')

In [154]:
ta_crossroads_cp = ta_crossroads.copy()
ta_crossroads_cp.geometry = ta_crossroads_cp.buffer(1.5, cap_style='square')

In [155]:
# ## checking created polygons

# m = leafmap.Map(center=(32.047, 34.785), zoom=11)
# # m.add_gdf(ta_crossroads_cp, fill_colors='black')
# m.add_gdf(os_ta_streets_edges_cp)


# m

**Below Slow code: about 10 to 40 min** 

In [156]:
import geopandas as gpd
from shapely.ops import unary_union

# 1. Union all crossroad geometries into a single geometry
crossroads_union = unary_union(ta_crossroads_cp.geometry)

# 2. Subtract that union from each edge’s geometry
#    This might yield MultiLineString if the line gets cut in multiple pieces
os_ta_streets_edges_cp["geometry"] = os_ta_streets_edges_cp.geometry.apply(
    lambda line: line.difference(crossroads_union)
)

# 3. Optional: explode MultiLineStrings into individual features
ta_streets_no_cr = os_ta_streets_edges_cp.explode(index_parts=True).reset_index(drop=True)

# 4. If needed, filter out any empty geometries that remain
ta_streets_no_cr = ta_streets_no_cr[~ta_streets_no_cr.geometry.is_empty]
ta_streets_no_cr


,os_ta_index,u,v,osmid,name,reversed,length,tunnel,bridge,junction,...,ta_name,overlapping_names,overlapping_osmids,overlapping_names_len,start_edge_idx,end_edge_idx,start_name,end_name,is_connected_to_rab,geometry
1,1,139693,139698,167691710,יהודה המכבי,False,62.173,,,,...,יהודה המכבי,,,0,0,2,ויצמן,יהודה המכבי,0,"POLYGON ((668980.518 3552247.213, 669017.739 3..."
2,2,139698,139723,167691710,יהודה המכבי,False,110.029,,,,...,יהודה המכבי,,,0,1,23,יהודה המכבי,יהודה המכבי,0,"POLYGON ((669082.169 3552255.902, 669126.926 3..."
3,3,139707,139708,26516058,ירמיהו,False,37.249,,,,...,ירמיהו הנביא,,,0,4,5,אוסישקין,ירמיהו הנביא,0,"POLYGON ((667810.374 3552369.672, 667815.386 3..."
4,4,139707,10985355495,1183058410,אוסישקין,False,190.227,,,,...,אוסישקין,,,0,3,9372,ירמיהו הנביא,אוסישקין,0,"POLYGON ((667840.623 3552390.785, 667843.963 3..."
5,5,139708,340368898,492978628,ירמיהו,False,90.699,,,,...,ירמיהו הנביא,,,0,3,1091,ירמיהו הנביא,ירמיהו הנביא,0,"POLYGON ((667732.696 3552349.366, 667797.488 3..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12526,9523,12287814424,544561044,"[396627330, 396627331, 396627332, 139744099, 1...",איילון דרום,False,1158.0299999999997,,yes,,...,איילון דרום,,,0,93,4375,איילון דרום,חיל השריון,0,"POLYGON ((668475.141 3547855.554, 668472.615 3..."
12527,9523,12287814424,544561044,"[396627330, 396627331, 396627332, 139744099, 1...",איילון דרום,False,1158.0299999999997,,yes,,...,איילון דרום,,,0,93,4375,איילון דרום,חיל השריון,0,"POLYGON ((668464.425 3547335.197, 668463.503 3..."
12528,9524,12287814424,2213119636,1082074722,,False,656.7250000000001,,,,...,"איילון דרום, כביש ירושלים תל אביב",,,0,93,96,איילון דרום,כביש ירושלים תל אביב,0,"POLYGON ((668473.907 3547727.631, 668472.730 3..."
12529,9525,12292683830,1790662878,167691679,המסגר,False,136.733,,,,...,המסגר,,,0,500,6863,המסגר,המסגר,0,"POLYGON ((668463.350 3548541.128, 668458.426 3..."


In [157]:
## checking created polygons

m = leafmap.Map(center=(32.047, 34.785), zoom=11)
m.add_gdf(ta_crossroads_cp,fill_colors='black')
m.add_gdf(ta_streets_no_cr)


m

Map(center=[32.047, 34.785], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom…

### Checking how significant will it be accident wise to drop some parts of streets.

In [158]:
ta_streets_no_cr['area'] = ta_streets_no_cr.geometry.area

In [159]:
ta_streets_no_cr[(ta_streets_no_cr['area'] > 30) & ( ta_streets_no_cr['area'] < 45)].shape

(56, 24)

In [160]:
# Exclude micro mobility
BICYCLE = 15
SCOOTER = 21
E_BICYCLE = 23
micro_m = [SCOOTER, E_BICYCLE, BICYCLE]


# Load original accident data
i_m_h_ta_gdf = gpd.read_parquet('./csv_tables/i_m_h_ta_gdf.parquet')    
#### Removing 2024 from **i_m_h_ta_gdf**
i_m_h_ta_gdf = i_m_h_ta_gdf[~(i_m_h_ta_gdf.accident_year == 2024)].copy()
# Accidents that are not MM
i_m_h_ta_no_mm_gdf = i_m_h_ta_gdf[~(i_m_h_ta_gdf.involve_vehicle_type.isin(micro_m))].copy()
# Accidents that are just MM
i_m_h_ta_mm_gdf =  gpd.read_parquet('./csv_tables/i_m_h_ta_mm_gdf.parquet')
display(i_m_h_ta_mm_gdf.crs)
i_m_h_ta_mm_gdf.head(5)

<Projected CRS: EPSG:32636>
Name: WGS 84 / UTM zone 36N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Between 30°E and 36°E, northern hemisphere between equator and 84°N, onshore and offshore. Belarus. Cyprus. Egypt. Ethiopia. Finland. Israel. Jordan. Kenya. Lebanon. Moldova. Norway. Russian Federation. Saudi Arabia. Sudan. Syria. Türkiye (Turkey). Uganda. Ukraine.
- bounds: (30.0, 0.0, 36.0, 84.0)
Coordinate Operation:
- name: UTM zone 36N
- method: Transverse Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

,accident_id,provider_and_id,provider_code,file_type_police,involved_type,involved_type_hebrew,license_acquiring_date,age_group,age_group_hebrew,sex,...,vehicle_attribution,vehicle_attribution_hebrew,seats,total_weight,total_weight_hebrew,vehicle_damage,vehicle_damage_hebrew,urban_intersection,accident_date,geometry
41,2013001368,32013001368,3,3,2,נהג נפגע,0,6,25-29,1,...,1.0,ישראלי,99.0,0.0,לא ידוע,4.0,אין נזק,NaN,2013-07-27 01:00:00,POINT (667544.749 3549959.961)
50,2013001742,32013001742,3,3,2,נהג נפגע,0,7,30-34,2,...,1.0,ישראלי,99.0,0.0,לא ידוע,4.0,אין נזק,NaN,2013-10-07 01:30:00,POINT (667286.918 3548726.540)
131,2013001350,12013001350,1,1,2,נהג נפגע,0,5,20-24,1,...,1.0,ישראלי,99.0,0.0,לא ידוע,4.0,אין נזק,NaN,2013-08-25 01:00:00,POINT (667023.688 3548785.170)
196,2013000147,12013000147,1,1,2,נהג נפגע,0,8,35-39,1,...,1.0,ישראלי,99.0,0.0,לא ידוע,2.0,בינוני,9110323.0,2013-09-19 00:00:00,POINT (668158.751 3551284.678)
197,2013000147,12013000147,1,1,2,נהג נפגע,0,6,25-29,1,...,1.0,ישראלי,99.0,0.0,לא ידוע,2.0,בינוני,9110323.0,2013-09-19 00:00:00,POINT (668158.751 3551284.678)


In [161]:
i_m_h_ta_gdf_cp = i_m_h_ta_gdf.copy()
i_m_h_ta_gdf_cp.geometry = i_m_h_ta_gdf_cp.buffer(4)

In [162]:
# ta_streets_no_cr.drop(columns=['index'], inplace=True)

In [163]:
ta_streets_no_cr.reset_index(inplace=True)
ta_streets_no_cr.columns = ['st_seg_idx', 'os_ta_index', 'u', 'v', 'osmid', 'name', 'reversed', 'length',
       'tunnel', 'bridge', 'junction', 'name_type', 'osmid_type', 'name_fixed',
       'ta_name', 'overlapping_names', 'overlapping_osmids',
       'overlapping_names_len', 'start_edge_idx', 'end_edge_idx', 'start_name',
       'end_name', 'is_connected_to_rab', 'geometry','area']

In [164]:
accidents_intersects = gpd.sjoin(i_m_h_ta_gdf_cp[i_m_h_ta_gdf_cp.location_accuracy == 1], ta_streets_no_cr[(ta_streets_no_cr['area'] > 45) & ( ta_streets_no_cr['area'] < 60)], how='inner', predicate='intersects')
seg_intersects_45    = gpd.sjoin(ta_streets_no_cr[(ta_streets_no_cr['area'] > 45) & ( ta_streets_no_cr['area'] < 60)], i_m_h_ta_gdf_cp[i_m_h_ta_gdf_cp.location_accuracy == 1],  how='inner', predicate='intersects')
accidents_intersects

,accident_id,provider_and_id,provider_code,file_type_police,involved_type,involved_type_hebrew,license_acquiring_date,age_group,age_group_hebrew,sex,...,ta_name,overlapping_names,overlapping_osmids,overlapping_names_len,start_edge_idx,end_edge_idx,start_name,end_name,is_connected_to_rab,area
647,2013003402,12013003402,1,1,2,נהג נפגע,1976,12,55-59,1,...,סנה משה,,,0,2337,265,סנה משה,קרית שאול,0,48.816603
1760,2013003402,12013003402,1,1,1,נהג,1951,18,85+,1,...,סנה משה,,,0,2337,265,סנה משה,קרית שאול,0,48.816603
3296,2013019382,12013019382,1,1,2,נהג נפגע,1995,8,35-39,1,...,עין יעקב,,,0,5876,4279,עין יעקב,"חברת ש""ס",0,46.320265
10454,2013045578,32013045578,3,3,2,נהג נפגע,1991,9,40-44,1,...,אביאסף,,,0,4619,4618,קלמן,עברי,0,51.608645
12223,2013054730,12013054730,1,1,1,נהג,2004,7,30-34,2,...,עין יעקב,,,0,5876,4279,עין יעקב,"חברת ש""ס",0,46.320265
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101557,2022098383,12022098383,1,1,3,נפגע,0,5,20-24,2,...,עין יעקב,,,0,5876,4279,עין יעקב,"חברת ש""ס",0,46.320265
101593,2022098383,12022098383,1,1,1,נהג,2004,8,35-39,1,...,עין יעקב,,,0,5876,4279,עין יעקב,"חברת ש""ס",0,46.320265
101863,2023000404,12023000404,1,1,1,נהג,2022,4,15-19,2,...,סנה משה,,,0,2337,265,סנה משה,קרית שאול,0,48.816603
101911,2023000404,12023000404,1,1,2,נהג נפגע,0,7,30-34,1,...,סנה משה,,,0,2337,265,סנה משה,קרית שאול,0,48.816603


In [165]:
ta_streets_no_cr.columns

Index(['st_seg_idx', 'os_ta_index', 'u', 'v', 'osmid', 'name', 'reversed',
       'length', 'tunnel', 'bridge', 'junction', 'name_type', 'osmid_type',
       'name_fixed', 'ta_name', 'overlapping_names', 'overlapping_osmids',
       'overlapping_names_len', 'start_edge_idx', 'end_edge_idx', 'start_name',
       'end_name', 'is_connected_to_rab', 'geometry', 'area'],
      dtype='object')

In [166]:
seg_idx_to_keep = seg_intersects_45.drop_duplicates(subset=['st_seg_idx']).st_seg_idx.values
len(seg_idx_to_keep)

17

In [167]:
# ## checking accidents 
# m = leafmap.Map(center=(32.047, 34.785), zoom=11)
# # m.add_gdf(ta_streets_no_cr[(ta_streets_no_cr['area'] > 45) & ( ta_streets_no_cr['area'] < 60)] ,fill_colors='black')
# m.add_gdf(ta_crossroads_cp,fill_colors='black')
# m.add_gdf(accidents_intersects[['geometry','provider_and_id']],  style={
#             "color": "red",
#             "weight": 3,
#             "opacity": 1,
#             "fillColor": "red",
#             "fillOpacity": 0.3,
#         })



# m

#### Conclusion:
drop below area of 45

In [168]:
ta_streets_no_cr = ta_streets_no_cr[(ta_streets_no_cr['area'] > 45) | (ta_streets_no_cr.st_seg_idx.isin(seg_idx_to_keep))].copy()

In [169]:
ta_streets_no_cr.to_parquet('./csv_tables/ta_streets_no_cr_poly_22012025.parquet')

In [170]:
ta_crossroads_cp.to_parquet('./csv_tables/ta_crossroads_22012025.parquet')

### Getting segments length

Steps and reasons:

1. copy: this way we don't destroy our original dataset
2. buff by 0.5: buffing smaller or bigger can creates inaccurate length calculation, 
3. union of small buffed edges
4. union of created streets segments
5. intersect: this part can now calculate it's length  which is the perimeter.
5. create gdf with correct crs
6. explode: so we have different parts of the intersecting polygons
7. sjoin: to add the length we calculated we use this to correspond street segment with intersecting part.

In [172]:
ta_streets_no_cr = gpd.read_parquet('./csv_tables/ta_streets_no_cr_poly_22012025.parquet')
ta_crossroads_cp = gpd.read_parquet('./csv_tables/ta_crossroads_22012025.parquet')


In [174]:
# ## checking created polygons

# m = leafmap.Map(center=(32.047, 34.785), zoom=11)
# m.add_gdf(ta_crossroads_cp,fill_colors='black')
# m.add_gdf(ta_streets_no_cr)


# m

In [175]:
### create very small buffed edges

os_ta_streets_edges_small_buff = os_ta_streets_edges.copy()
os_ta_streets_edges_small_buff.geometry = os_ta_streets_edges_small_buff.buffer(0.5)

os_ta_streets_small_buff_union = unary_union(os_ta_streets_edges_small_buff.geometry)
ta_streets_no_cr_union = unary_union(ta_streets_no_cr.geometry)


# 2. Compute the intersection of these unions
intersection = ta_streets_no_cr_union.intersection(os_ta_streets_small_buff_union)

# 3. Wrap the intersection in a GeoDataFrame if you want to save or visualize
intersection_gdf = gpd.GeoDataFrame(geometry=[intersection], crs=os_ta_streets_edges.crs)


# # gdf
# os_ta_streets_small_buff_union_gdf = gpd.GeoDataFrame(geometry=[os_ta_streets_small_buff_union], crs=os_ta_streets_edges.crs)
os_ta_streets_small_buff_explode_gdf = intersection_gdf.explode(index_parts=True).reset_index(drop=True)

In [176]:
os_ta_streets_small_buff_explode_gdf['seg_length'] = round(os_ta_streets_small_buff_explode_gdf.geometry.length/2,3)

In [177]:
ta_streets_no_cr_sjoin = gpd.sjoin(ta_streets_no_cr, os_ta_streets_small_buff_explode_gdf, how='inner',predicate='intersects')
print(ta_streets_no_cr_sjoin.shape)
ta_streets_no_cr_sjoin = ta_streets_no_cr_sjoin.drop_duplicates(subset=['index_right'])
print(ta_streets_no_cr_sjoin.shape)
ta_streets_no_cr_sjoin = ta_streets_no_cr_sjoin.drop_duplicates(subset=['st_seg_idx'])
print(ta_streets_no_cr_sjoin.shape)


(6927, 27)
(6355, 27)
(6116, 27)


In [178]:
ta_streets_no_cr_sjoin['seg_length_2'] = round(ta_streets_no_cr_sjoin.geometry.length / 2, 3)
ta_streets_no_cr_sjoin['seg_length_2']

0         50.637
1         98.206
2         25.138
3        178.549
4         78.833
          ...   
10811    123.137
10813     85.568
10819    182.128
10823    515.383
10827     74.361
Name: seg_length_2, Length: 6116, dtype: float64

In [179]:
ta_streets_no_cr_sjoin[ta_streets_no_cr_sjoin.seg_length > ta_streets_no_cr_sjoin.seg_length_2].shape

(48, 28)

### After checking differences in length we found some issues.

* some segments that have been expanded by 0.5 (the small ones) were later combined in the sjoin so their length represents it.
* some sjoin capture more then 1 small segments and takes it's length.

Plan:
* We will drop segments have their seg_length > seg_length_2
* the rest get the difference between seg_length and seg_length_2
* check if the difference is the same or close to the same, if so:
    * find the avg of the difference and remove it from seg_length_2
* change ta_streets_no_cr_sjoin seg_length to use geometry.length - the difference



In [180]:
mask = ta_streets_no_cr_sjoin[ta_streets_no_cr_sjoin.seg_length < ta_streets_no_cr_sjoin.seg_length_2].st_seg_idx
ta_streets_no_cr_sjoin_cp = ta_streets_no_cr_sjoin[ta_streets_no_cr_sjoin.st_seg_idx.isin(mask)].copy()
ta_streets_no_cr_sjoin_cp.shape

(6068, 28)

In [181]:
ta_streets_no_cr_sjoin_cp['seg_diff'] = ta_streets_no_cr_sjoin_cp['seg_length_2'] - ta_streets_no_cr_sjoin_cp['seg_length']
print(ta_streets_no_cr_sjoin_cp['seg_diff'].quantile(0.01))
print(ta_streets_no_cr_sjoin_cp['seg_diff'].quantile(0.05))
print(ta_streets_no_cr_sjoin_cp['seg_diff'].quantile(0.10))
print(ta_streets_no_cr_sjoin_cp['seg_diff'].quantile(0.25))
print(ta_streets_no_cr_sjoin_cp['seg_diff'].quantile(0.5))
print(ta_streets_no_cr_sjoin_cp['seg_diff'].quantile(0.75))
print(ta_streets_no_cr_sjoin_cp['seg_diff'].quantile(0.90))
print(ta_streets_no_cr_sjoin_cp['seg_diff'].quantile(0.95))
print(ta_streets_no_cr_sjoin_cp['seg_diff'].quantile(0.99))

11.168360000000003
12.397
12.855700000000006
12.959999999999994
12.960000000000008
13.261500000000003
18.694999999999997
18.695999999999998
36.29313999999997


Conclusion: remove 14

In [182]:
ta_streets_no_cr_sjoin_cp.drop(columns=['seg_length','seg_diff'], inplace=True)

In [183]:
ta_streets_no_cr_sjoin_cp.columns


Index(['st_seg_idx', 'os_ta_index', 'u', 'v', 'osmid', 'name', 'reversed',
       'length', 'tunnel', 'bridge', 'junction', 'name_type', 'osmid_type',
       'name_fixed', 'ta_name', 'overlapping_names', 'overlapping_osmids',
       'overlapping_names_len', 'start_edge_idx', 'end_edge_idx', 'start_name',
       'end_name', 'is_connected_to_rab', 'geometry', 'area', 'index_right',
       'seg_length_2'],
      dtype='object')

In [184]:
ta_streets_no_cr_sjoin_cp.columns = ['st_seg_idx', 'os_ta_index', 'u', 'v', 'osmid', 'name', 'reversed',
       'length', 'tunnel', 'bridge', 'junction', 'name_type', 'osmid_type',
       'name_fixed', 'ta_name', 'overlapping_names', 'overlapping_osmids',
       'overlapping_names_len', 'start_edge_idx', 'end_edge_idx', 'start_name',
       'end_name', 'is_connected_to_rab', 'geometry', 'area', 'index_right',
       'seg_length']

In [185]:
ta_streets_no_cr_sjoin_cp[ta_streets_no_cr_sjoin_cp.seg_length < 14]

,st_seg_idx,os_ta_index,u,v,osmid,name,reversed,length,tunnel,bridge,...,overlapping_names_len,start_edge_idx,end_edge_idx,start_name,end_name,is_connected_to_rab,geometry,area,index_right,seg_length


In [186]:
ta_streets_no_cr_sjoin_cp.seg_length = ta_streets_no_cr_sjoin_cp.seg_length - 14

In [187]:
# #### Checking if length make sense

# m = leafmap.Map(center=(32.047, 34.785), zoom=11)
# # m.add_gdf(ta_streets_no_cr_sjoin_cp[['st_seg_idx','seg_length','seg_length_2','geometry']], fill_colors='black')
# m.add_gdf(ta_streets_no_cr_sjoin_cp[['seg_length','geometry']]
# , fill_colors='black')
# # m.add_gdf(os_ta_streets_small_buff_explode_gdf[['geometry','seg_length']])



# m

# ### it does

Seems to work well

In [188]:
col_to_keep = ['st_seg_idx', 'os_ta_index', 'u', 'v', 'osmid', 'name', 'reversed',
       'length', 'tunnel', 'bridge', 'junction', 'name_type', 'osmid_type',
       'name_fixed', 'ta_name', 'overlapping_names', 'overlapping_osmids',
       'overlapping_names_len', 'start_edge_idx', 'end_edge_idx', 'start_name',
       'end_name', 'is_connected_to_rab', 'geometry', 'area',
       'seg_length']

In [189]:
ta_streets_no_cr_sjoin[col_to_keep].to_parquet('./csv_tables/ta_streets_no_cr_poly_22012025.parquet')

Handling specific seg and cr

cr_idx = [2156]
sg_idx = [76]

In [200]:
ta_crossroads    = gpd.read_parquet('./csv_tables/ta_crossroads_22012025.parquet')
ta_streets_no_cr = gpd.read_parquet('./csv_tables/ta_streets_no_cr_poly_22012025.parquet')


In [201]:
## checking created polygons

m = leafmap.Map(center=(32.047, 34.785), zoom=11)
m.add_gdf(ta_crossroads,fill_colors='black')
m.add_gdf(ta_streets_no_cr)


m

Map(center=[32.047, 34.785], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom…

In [202]:
geo_1 = ta_crossroads[ta_crossroads.cr_idx == 1745].geometry
geo_2   = ta_streets_no_cr[ta_streets_no_cr.st_seg_idx == 74].geometry.buffer(4)

poly1 = geo_1.iloc[0]  # first (and only) geometry in geo_2156
poly2 = geo_2.iloc[0]    # first (and only) geometry in geo_76


df_combined = pd.DataFrame([poly1, poly2])
df_combined.columns = ['geometry']
before_combined = gpd.GeoDataFrame(df_combined, crs=ta_crossroads.crs)

# Combine into a single geometry
combined = unary_union(before_combined.geometry)

# Clean up internal boundaries or invalid rings

# Assign it back
mask = ta_crossroads[ta_crossroads.cr_idx == 2156].index
ta_crossroads.loc[mask, 'geometry'] = combined


In [204]:
#### Checking if length make sense

m = leafmap.Map(center=(32.047, 34.785), zoom=11)

m.add_gdf(ta_crossroads)
# m.add_gdf(ta_streets_no_cr, fill_colors='black')

m

Map(center=[32.047, 34.785], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom…

In [206]:
ta_streets_no_cr.shape

(6116, 26)

In [207]:
ta_streets_no_cr = ta_streets_no_cr[ta_streets_no_cr.st_seg_idx != 74]
ta_streets_no_cr.shape

(6115, 26)

In [208]:
ta_streets_no_cr.to_parquet('./csv_tables/ta_streets_no_cr_poly_22012025.parquet')
ta_crossroads.to_parquet('./csv_tables/ta_crossroads_22012025.parquet')

### Checking crossroads ans streets segments and finding segments to add to crossroads

In [108]:
ta_streets_no_cr = gpd.read_parquet('./csv_tables/ta_streets_no_cr_poly_20012025.parquet')
ta_crossroads = gpd.read_parquet('./csv_tables/ta_crossroads_20012025.parquet')


In [109]:
ta_streets_no_cr['seg_length'] = round((ta_streets_no_cr.geometry.length / 2) - 14, 3)

In [110]:
ta_streets_no_cr_cp = ta_streets_no_cr.copy()
ta_streets_no_cr_cp.geometry = ta_streets_no_cr.buffer(3)

In [ ]:
ta_streets_no_cr_cp_sjoin_cr = gpd.sjoin(ta_streets_no_cr_cp, ta_crossroads, how='inner', predicate='intersects')
cr_sjoin_ta_streets_no_cr_cp = gpd.sjoin(ta_crossroads, ta_streets_no_cr_cp , how='inner', predicate='intersects')

display(ta_streets_no_cr_cp_sjoin_cr.shape)
cr_sjoin_ta_streets_no_cr_cp.shape

(12014, 34)

(12014, 34)

In [112]:
gb_sg = ta_streets_no_cr_cp_sjoin_cr[ta_streets_no_cr_cp_sjoin_cr.seg_length < 40].groupby('st_seg_idx').count()
sg_to_check = gb_sg[gb_sg.name > 1].index

cr_to_check = cr_sjoin_ta_streets_no_cr_cp[cr_sjoin_ta_streets_no_cr_cp.st_seg_idx.isin(sg_to_check)].index
display(cr_to_check.shape)
sg_to_check.shape

(3846,)

(1905,)

In [116]:
#### Checking if length make sense

m = leafmap.Map(center=(32.047, 34.785), zoom=11)
m.add_gdf(ta_crossroads[ta_crossroads.cr_idx.isin(cr_to_check)][['cr_idx','geometry']],)
m.add_gdf(ta_streets_no_cr[ta_streets_no_cr.st_seg_idx.isin([9125, 9768,11468 ])][['st_seg_idx','geometry','seg_length']], fill_colors='black')
m
